In this notebook I will create a training table equivalent to ml_model_run_details

# Define Library

In [2]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
import tempfile
import time
from datetime import datetime
import uuid
import joblib
import uuid

import gcsfs
import duckdb as dd
import pickle
import joblib
from typing import Union
import io
path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)

### Function

#### expand_calc_features

In [3]:
import pandas as pd
import json

def expand_calc_features(df):
    """
    Expand the calcFeatures JSON column into separate columns and return the complete DataFrame.

    Parameters:
    df (pd.DataFrame): Input DataFrame with calcFeatures column containing JSON data

    Returns:
    pd.DataFrame: Expanded DataFrame with all original columns plus JSON features as separate columns
    """

    # Make a copy to avoid modifying the original DataFrame
    df_expanded = df.copy()

    # Parse the calcFeatures JSON column
    calc_features_list = []

    for idx, calc_features_str in enumerate(df['calcFeatures']):
        try:
            # Parse the JSON string
            features_dict = json.loads(calc_features_str.replace("'", '"'))  # Replace single quotes with double quotes for valid JSON
            calc_features_list.append(features_dict)
        except (json.JSONDecodeError, AttributeError) as e:
            # If parsing fails, create an empty dict and print warning
            print(f"Warning: Could not parse calcFeatures at index {idx}: {e}")
            calc_features_list.append({})

    # Create DataFrame from the parsed JSON data
    calc_features_df = pd.DataFrame(calc_features_list)

    # Add prefix to JSON-derived columns to avoid conflicts
    calc_features_df = calc_features_df.add_prefix('calc_')

    # Reset index to ensure proper alignment
    df_expanded = df_expanded.reset_index(drop=True)
    calc_features_df = calc_features_df.reset_index(drop=True)

    # Combine original DataFrame with expanded calcFeatures
    result_df = pd.concat([df_expanded, calc_features_df], axis=1)

    return result_df


#### expand_calc_features_robust

In [4]:
import pandas as pd
import json

def expand_calc_features_robust(df):
    """
    Expand the calcFeatures JSON column into separate columns with better error handling.

    Parameters:
    df (pd.DataFrame): Input DataFrame with calcFeatures column containing JSON data

    Returns:
    pd.DataFrame: Expanded DataFrame with all original columns plus JSON features as separate columns
    """

    # Make a copy to avoid modifying the original DataFrame
    df_expanded = df.copy()

    # Parse the calcFeatures JSON column
    calc_features_data = []

    for idx, row in df.iterrows():
        calc_features_str = row['calcFeatures']

        if pd.isna(calc_features_str) or calc_features_str == '':
            calc_features_data.append({})
            continue

        try:
            # Clean the string and parse JSON
            cleaned_str = calc_features_str.replace("'", '"').replace('None', 'null').replace('True', 'true').replace('False', 'false')
            features_dict = json.loads(cleaned_str)
            calc_features_data.append(features_dict)
        except Exception as e:
            print(f"Warning: Could not parse calcFeatures at index {idx}: {e}")
            print(f"Problematic string: {calc_features_str[:100]}...")  # Print first 100 chars
            calc_features_data.append({})

    # Create DataFrame from the parsed JSON data
    calc_features_df = pd.DataFrame(calc_features_data)

    # Add prefix to JSON-derived columns to avoid conflicts with existing columns
    calc_features_df = calc_features_df.add_prefix('feat_')

    # Combine DataFrames
    result_df = pd.concat([df_expanded, calc_features_df], axis=1)

    print(f"Original DataFrame shape: {df.shape}")
    print(f"Expanded DataFrame shape: {result_df.shape}")
    print(f"Added {len(calc_features_df.columns)} new columns from calcFeatures")

    return result_df

#### transform_data

In [5]:
# import pandas as pd
# import json
# import uuid
# from datetime import datetime
# from typing import List

# def transform_data(d1: pd.DataFrame, feature_column: List[str], a='demo_score', modelDisplayName = 'Cash_beta_trench1_Demo_backscore', subscription_name = 'sil_march 25 models'):
#     # Read the input CSV file
#     df = d1.copy()
    
#     # Create the output DataFrame with the required structure
#     output_data = []
    
#     for _, row in df.iterrows():
#         # Create the calcFeature JSON with all the feature columns
#         feature_columns = feature_column
        
#         calc_feature = {}
#         for col in feature_columns:
#             if col in row and pd.notna(row[col]):
#                 # Convert Timestamp objects to string
#                 if isinstance(row[col], pd.Timestamp):
#                     calc_feature[col] = row[col].isoformat()
#                 else:
#                     calc_feature[col] = row[col]
        
       
#         # Get current timestamp
#         current_time = datetime.now().isoformat()
        
#         # Create the output row
#         output_row = {
#             "customerId": row['customer_id'],
#             "digitalLoanAccountId": row['digitalLoanAccountId'],
#             "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
#             "prediction": row.get(a, 0),
#             "start_time": current_time,
#             "end_time": current_time,
#             "modelDisplayName":modelDisplayName,
#             "modelVersionId":"v1",
#             "subscription_name": subscription_name,
#             "message_id": str(uuid.uuid4()),  # Generate random UUID
#             "publish_time": current_time,
#             "attributes": "{}",  # Empty JSON object
#             "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
#         }
        
#         output_data.append(output_row)
    
#     # Create DataFrame from the output data
#     output_df = pd.DataFrame(output_data)
    
#     return output_df

# # Example usage:
# # transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# # transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [6]:
import pandas as pd
import json
import uuid
from datetime import datetime
from typing import List

def transform_data(
    d1: pd.DataFrame, 
    feature_column: List[str], 
    a: str = 'demo_score', 
    modelDisplayName: str = 'Cash_beta_trench1_Demo_backscore', 
    tc: str = "", 
    subscription_name: str = 'sil_march 25 models'
) -> pd.DataFrame:
    """
    Transforms input data into a structured format suitable for model scoring output.

    Parameters:
    - d1 (pd.DataFrame): Input DataFrame containing raw data.
    - feature_column (List[str]): List of column names to include in the 'calcFeature' JSON.
    - a (str): Column name containing the prediction score. Default is 'demo_score'.
    - modelDisplayName (str): Name of the model used for scoring.
    - tc (str): Trench category (optional).
    - do (str): Device operating system. Default is 'android'.
    - subscription_name (str): Name of the subscription or model group.

    Returns:
    - pd.DataFrame: Transformed DataFrame with structured output.
    """

    # Make a copy of the input DataFrame to avoid modifying the original
    df = d1.copy()
    
    # Initialize an empty list to store transformed rows
    output_data = []
    
    # Iterate over each row in the DataFrame
    for _, row in df.iterrows():
        # Initialize dictionary to hold feature values
        calc_feature = {}
        
        # Loop through each feature column and extract its value from the row
        for col in feature_column:
            if col in row and pd.notna(row[col]):
                # Convert datetime values to ISO format strings
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
        # Get the current timestamp for start_time, end_time, and publish_time
        current_time = datetime.now().isoformat()
        
        # Construct the output row dictionary with required fields
        output_row = {
            "customerId": row['customer_id'],  # Unique customer identifier
            "digitalLoanAccountId": row['digitalLoanAccountId'],  # Loan account ID
            "crifApplicationId": str(uuid.uuid4()),  # Random UUID for application ID
            "prediction": row.get(a, 0),  # Prediction score from specified column
            "start_time": current_time,  # Timestamp when processing starts
            "end_time": current_time,    # Timestamp when processing ends
            "modelDisplayName": modelDisplayName,  # Name of the model used
            "modelVersionId": "v1",  # Static model version
            "calcFeature": json.dumps(calc_feature, default=str),  # Features as JSON string
            "subscription_name": subscription_name,  # Subscription name
            "message_id": str(uuid.uuid4()),  # Random UUID for message ID
            "publish_time": current_time,  # Timestamp when message is published
            "attributes": "{}",  # Placeholder for additional attributes
            "trenchCategory": tc,  # Optional trench category
            "deviceOs": row['osType'],  # Device operating system
        }
        
        # Append the transformed row to the output list
        output_data.append(output_row)
    
    # Convert the list of dictionaries to a DataFrame
    output_df = pd.DataFrame(output_data)
    
    # Return the transformed DataFrame
    return output_df


#### PSI Functions new

In [7]:
## Updated on 27-10-2025 - Modified for Training Period Baseline
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

def identify_feature_types(df: pd.DataFrame, feature_list: List[str]) -> Dict[str, List[str]]:
    """
    Identify categorical and numerical features from the feature list.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to classify

    Returns:
    --------
    Dict with 'categorical' and 'numerical' keys containing respective feature lists
    """
    categorical_features = []
    numerical_features = []

    for feature in feature_list:
        if feature not in df.columns:
            print(f"Warning: Feature '{feature}' not found in dataframe")
            continue

        # Check if feature is numeric
        if pd.api.types.is_numeric_dtype(df[feature]):
            # If unique values are less than 15 and all integers, treat as categorical
            unique_vals = df[feature].nunique()
            if unique_vals < 15 and df[feature].dropna().apply(lambda x: x == int(x) if isinstance(x, (int, float)) else False).all():
                categorical_features.append(feature)
            else:
                numerical_features.append(feature)
        else:
            categorical_features.append(feature)

    return {
        'categorical': categorical_features,
        'numerical': numerical_features
    }


def create_bins_for_features(df: pd.DataFrame,
                             numerical_features: List[str],
                             categorical_features: List[str],
                             train_period_df: pd.DataFrame) -> Dict:
    """
    Create bins for numerical features (deciles with fallback) and categorical features (top 6 + others)
    based on the entire training period data.

    Parameters:
    -----------
    df : pd.DataFrame
        Full input dataframe
    numerical_features : List[str]
        List of numerical features
    categorical_features : List[str]
        List of categorical features
    train_period_df : pd.DataFrame
        Training period dataframe (June 2024 to March 2025)

    Returns:
    --------
    Dictionary containing binning information for each feature
    """
    binning_info = {}

    # Create bins for numerical features with fallback strategy
    for feature in numerical_features:
        valid_data = train_period_df[feature].dropna()

        if len(valid_data) == 0:
            binning_info[feature] = {'type': 'numerical', 'bins': None, 'bin_ranges': {}}
            continue

        bins = None
        bin_count = None

        # Try 10 bins (deciles)
        try:
            test_bins = np.percentile(valid_data, np.arange(0, 101, 10))
            test_bins = np.unique(test_bins)
            if len(test_bins) >= 11:  # 11 edges = 10 bins
                bins = test_bins
                bin_count = 10
        except Exception as e:
            pass

        # If 10 bins not possible, try 5 bins
        if bins is None:
            try:
                test_bins = np.percentile(valid_data, np.arange(0, 101, 20))
                test_bins = np.unique(test_bins)
                if len(test_bins) >= 6:  # 6 edges = 5 bins
                    bins = test_bins
                    bin_count = 5
            except Exception as e:
                pass

        # If 5 bins not possible, try 3 bins
        if bins is None:
            try:
                test_bins = np.percentile(valid_data, [0, 33.33, 66.67, 100])
                test_bins = np.unique(test_bins)
                if len(test_bins) >= 4:  # 4 edges = 3 bins
                    bins = test_bins
                    bin_count = 3
            except Exception as e:
                pass

        # If still no bins possible, use equal distance bins of 5
        if bins is None:
            print(f"Warning: Feature '{feature}' has insufficient variance - cannot create standard bins")
            print(f"Feature '{feature}': Using equal distance bins of 5")

            min_val = valid_data.min()
            max_val = valid_data.max()

            # Create 5 equal distance bins
            bins = np.linspace(min_val, max_val, 6)  # 6 edges = 5 bins
            bins = np.unique(bins)
            bin_count = len(bins) - 1

            # If all values are the same, add slight buffer
            if bin_count == 1:
                bins = np.array([min_val - 0.1, min_val, min_val + 0.1])
                bin_count = 2
                print(f"Feature '{feature}': Constant value ({min_val}). Created 2 equal distance bins with buffer")

        # Add infinity edges to capture all values
        bins = bins.copy()
        bins[0] = -np.inf
        bins[-1] = np.inf

        print(f"Feature '{feature}': Created {bin_count} bins")

        # Create bin ranges dictionary
        bin_ranges = {}
        for i in range(len(bins)-1):
            bin_name = f"Bin_{i+1}"
            bin_ranges[bin_name] = {
                'min': bins[i],
                'max': bins[i+1],
                'range_str': f"[{bins[i]:.2f}, {bins[i+1]:.2f}]" if not np.isinf(bins[i]) and not np.isinf(bins[i+1]) else f"({bins[i]}, {bins[i+1]})"
            }

        binning_info[feature] = {
            'type': 'numerical',
            'bins': bins,
            'bin_ranges': bin_ranges,
            'bin_count': bin_count
        }

    # Create bins for categorical features (top 6 + others) using training period
    for feature in categorical_features:
        value_counts = train_period_df[feature].value_counts()
        unique_categories = value_counts.index.tolist()
        print(f"Unique categories: {unique_categories}")

        if len(unique_categories) <= 6:
            # Treat each category as a separate bin
            top_categories = unique_categories
        else:
            # Use top 6 categories only
            top_categories = value_counts.nlargest(6).index.tolist()

        print(f"Top categories for feature '{feature}': {top_categories}")

        binning_info[feature] = {
                'type': 'categorical',
                'top_categories': top_categories,
                'bin_ranges': {}  # No ranges for categorical
            }

    return binning_info


def apply_binning(df: pd.DataFrame,
                  feature: str,
                  binning_info: Dict) -> pd.Series:
    """
    Apply binning to a feature based on binning information.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature : str
        Feature name
    binning_info : Dict
        Binning information for the feature

    Returns:
    --------
    pd.Series with binned values
    """
    if binning_info['type'] == 'numerical':
        if binning_info['bins'] is None:
            return pd.Series(['Missing'] * len(df), index=df.index)

        bins = binning_info['bins']
        labels = [f"Bin_{i+1}" for i in range(len(bins)-1)]

        binned = pd.cut(df[feature],
                       bins=bins,
                       labels=labels,
                       include_lowest=True,
                       duplicates='drop')

        # Handle nulls - convert to string and then replace
        binned = binned.astype(str)
        binned[df[feature].isna()] = 'Missing'

        return binned

    else:  # categorical
        top_cats = binning_info['top_categories']

        # Convert to string for consistent comparison
        if pd.api.types.is_categorical_dtype(df[feature]):
            feature_data = df[feature].astype(str)
        else:
            feature_data = df[feature].astype(str)

        # Replace NaN string representation with 'Missing'
        feature_data = feature_data.replace('nan', 'Missing')

        # Convert top_cats to strings for comparison
        top_cats_str = [str(cat) for cat in top_cats]

        # Apply binning logic: use category name if in top_cats, else 'Others' (except for Missing)
        binned = feature_data.apply(lambda x: x if x in top_cats_str else ('Others' if x != 'Missing' else 'Missing'))

        return binned


def calculate_psi(expected_pct: pd.Series,
                  actual_pct: pd.Series,
                  epsilon: float = 0.0001) -> float:
    """
    Calculate Population Stability Index with proper epsilon handling and renormalization.

    Parameters:
    -----------
    expected_pct : pd.Series
        Expected (baseline) percentages
    actual_pct : pd.Series
        Actual percentages
    epsilon : float
        Small value to avoid log(0)

    Returns:
    --------
    PSI value
    """
    # Align indices
    all_bins = expected_pct.index.union(actual_pct.index)
    expected_pct = expected_pct.reindex(all_bins, fill_value=0)
    actual_pct = actual_pct.reindex(all_bins, fill_value=0)

    # Only add epsilon where values are zero
    expected_pct = expected_pct.apply(lambda x: epsilon if x == 0 else x)
    actual_pct = actual_pct.apply(lambda x: epsilon if x == 0 else x)

    # Renormalize to ensure they sum to 1 after adding epsilon
    expected_pct = expected_pct / expected_pct.sum()
    actual_pct = actual_pct / actual_pct.sum()

    # Calculate PSI
    psi_value = np.sum((actual_pct - expected_pct) * np.log(actual_pct / expected_pct))

    return psi_value


def calculate_month_on_month_psi(df: pd.DataFrame,
                                 feature_list: List[str],
                                 segment_columns: List[str],
                                 month_col: str = 'Application_month',
                                 data_selection_col: str = 'Data_selection',
                                 account_id_col: str = 'digitalLoanAccountId') -> pd.DataFrame:
    """
    Calculate PSI for each feature comparing training period (June 2024 to March 2025)
    vs each month after March 2025, overall and by segments.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to calculate PSI for
    segment_columns : List[str]
        List of segment columns
    month_col : str
        Name of month column
    data_selection_col : str
        Name of data selection column (identifies train period)
    account_id_col : str
        Name of account ID column for counting distinct accounts

    Returns:
    --------
    pd.DataFrame with PSI values with one row per feature-month-segment combination
    """
    # Create a copy to avoid modifying original
    df = df.copy()

    # Identify training and test periods
    train_df = df[df[data_selection_col] == 'Train'].copy()
    test_df = df[df[data_selection_col] != 'Train'].copy()

    if len(train_df) == 0:
        raise ValueError("No training data found. Check Data_selection column.")

    print(f"Training period: {train_df[month_col].min()} to {train_df[month_col].max()}")
    print(f"Test period: {test_df[month_col].min()} to {test_df[month_col].max()}")

    # Identify feature types
    feature_types = identify_feature_types(df, feature_list)

    # Create binning strategy based on training period
    binning_info = create_bins_for_features(
        df,
        feature_types['numerical'],
        feature_types['categorical'],
        train_df
    )

    # Get sorted test months
    test_months = sorted(test_df[month_col].unique())

    results = []

    # Calculate overall PSI
    for feature in feature_list:
        if feature not in df.columns:
            continue

        # Apply binning to entire dataset
        df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        # print(f"Feature binned {df[f'{feature}_binned']}")
        # Get training period distribution (baseline)
        train_baseline = df[df[data_selection_col] == 'Train'][f'{feature}_binned'].value_counts(normalize=True)

        # Calculate PSI for each test month
        for month in test_months:
            actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
            psi_value = calculate_psi(train_baseline, actual_dist)

            # Calculate average percentages across all bins
            expected_avg_pct = train_baseline.mean() * 100
            actual_avg_pct = actual_dist.mean() * 100

            # # Count distinct accounts for segment
            # base_segment_count = train_segment[account_id_col].nunique()
            # actual_segment_count = actual_segment[account_id_col].nunique()


            results.append({
                'Feature': feature,
                'Feature_Type': binning_info[feature]['type'],
                'Segment_Column': 'Overall',
                'Segment_Value': 'All',
                'Month': f"{month}",
                'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                'Current_Month': month,
                'Expected_Percentage': expected_avg_pct,
                'Actual_Percentage': actual_avg_pct,
                'PSI': psi_value
            })

    # Calculate PSI by segments
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            continue

        segments = df[segment_col].dropna().unique()

        for segment_val in segments:
            segment_df = df[df[segment_col] == segment_val]

            for feature in feature_list:
                if feature not in df.columns:
                    continue

                # Get training period distribution for segment
                train_segment = segment_df[segment_df[data_selection_col] == 'Train']
                if len(train_segment) == 0:
                    continue

                train_baseline = train_segment[f'{feature}_binned'].value_counts(normalize=True)

                # Calculate PSI for each test month
                for month in test_months:
                    actual_segment = segment_df[segment_df[month_col] == month]
                    if len(actual_segment) == 0:
                        continue

                    actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
                    psi_value = calculate_psi(train_baseline, actual_dist)

                    # Calculate average percentages across all bins
                    expected_avg_pct = train_baseline.mean() * 100
                    actual_avg_pct = actual_dist.mean() * 100

                    # Count distinct accounts for segment
                    base_segment_count = train_segment[account_id_col].nunique()
                    actual_segment_count = actual_segment[account_id_col].nunique()

                    results.append({
                        'Feature': feature,
                        'Feature_Type': binning_info[feature]['type'],
                        'Segment_Column': segment_col,
                        'Segment_Value': segment_val,
                        'Month': f"{month}",
                        'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                        'Current_Month': month,
                        'Base_Count': base_segment_count,
                        'Actual_Count': actual_segment_count,
                        'Expected_Percentage': expected_avg_pct,
                        'Actual_Percentage': actual_avg_pct,
                        'PSI': psi_value
                    })

    return pd.DataFrame(results)


def calculate_bin_level_psi(df: pd.DataFrame,
                            feature_list: List[str],
                            segment_columns: List[str],
                            month_col: str = 'Application_month',
                            data_selection_col: str = 'Data_selection',
                            account_id_col: str = 'digitalLoanAccountId') -> pd.DataFrame:
    """
    Calculate bin-level PSI for each feature comparing training period
    vs each month after March 2025, overall and by segments.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to calculate PSI for
    segment_columns : List[str]
        List of segment columns
    month_col : str
        Name of month column
    data_selection_col : str
        Name of data selection column
    account_id_col : str
        Name of account ID column for counting distinct accounts

    Returns:
    --------
    pd.DataFrame with bin-level PSI details including bin ranges
    """
    # Create a copy to avoid modifying original
    df = df.copy()

    # Identify training and test periods
    train_df = df[df[data_selection_col] == 'Train'].copy()
    test_df = df[df[data_selection_col] != 'Train'].copy()

    if len(train_df) == 0:
        raise ValueError("No training data found. Check Data_selection column.")

    print(f"Training period: {train_df[month_col].min()} to {train_df[month_col].max()}")
    print(f"Test period: {test_df[month_col].min()} to {test_df[month_col].max()}")

    # Identify feature types
    feature_types = identify_feature_types(df, feature_list)

    # Create binning strategy based on training period
    binning_info = create_bins_for_features(
        df,
        feature_types['numerical'],
        feature_types['categorical'],
        train_df
    )

    # Get sorted test months
    test_months = sorted(test_df[month_col].unique())

    results = []
    epsilon = 0.0001

    # Calculate overall bin-level PSI
    for feature in feature_list:
        if feature not in df.columns:
            continue

        # Apply binning to entire dataset
        df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        # print(df[f'{feature}_binned'])

        # Get training period distribution (baseline)
        train_baseline = df[df[data_selection_col] == 'Train'][f'{feature}_binned'].value_counts(normalize=True)

        # Calculate bin-level PSI for each test month
        for month in test_months:
            month_data = df[df[month_col] == month]
            actual_dist = month_data[f'{feature}_binned'].value_counts(normalize=True)

            # Count distinct accounts
            base_count = df[df[data_selection_col] == 'Train'][account_id_col].nunique()
            actual_count = month_data[account_id_col].nunique()

            # Get all bins
            all_bins = train_baseline.index.union(actual_dist.index)

            for bin_name in all_bins:
                # Simplified epsilon logic - no redundancy
                expected_pct = train_baseline.get(bin_name, 0)
                actual_pct = actual_dist.get(bin_name, 0)

                # Add epsilon only if zero
                expected_pct = epsilon if expected_pct == 0 else expected_pct
                actual_pct = epsilon if actual_pct == 0 else actual_pct

                # Calculate bin-level PSI
                bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct)

                # Get bin range information
                bin_ranges = binning_info[feature]['bin_ranges']
                if bin_name in bin_ranges:
                    bin_min = bin_ranges[bin_name]['min']
                    bin_max = bin_ranges[bin_name]['max']
                    bin_range = bin_ranges[bin_name]['range_str']
                else:
                    # For categorical or special bins (Missing, Others)
                    bin_min = None
                    bin_max = None
                    bin_range = bin_name

                results.append({
                    'Feature': feature,
                    'Feature_Type': binning_info[feature]['type'],
                    'Segment_Column': 'Overall',
                    'Segment_Value': 'All',
                    'Month': f"{month}",
                    'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                    'Current_Month': month,
                    'Base_Count': base_count,
                    'Actual_Count': actual_count,
                    'Bin': bin_name,
                    'Bin_Range': bin_range,
                    'Bin_Min': bin_min,
                    'Bin_Max': bin_max,
                    'Base_Percentage': (train_baseline.get(bin_name, 0) * 100),
                    'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
                    'Bin_PSI': bin_psi
                })

    # Calculate bin-level PSI by segments
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            continue

        segments = df[segment_col].dropna().unique()

        for segment_val in segments:
            segment_df = df[df[segment_col] == segment_val]

            for feature in feature_list:
                if feature not in df.columns:
                    continue

                # Get training period distribution for segment
                train_segment = segment_df[segment_df[data_selection_col] == 'Train']
                if len(train_segment) == 0:
                    continue

                train_baseline = train_segment[f'{feature}_binned'].value_counts(normalize=True)

                # Calculate bin-level PSI for each test month
                for month in test_months:
                    actual_segment = segment_df[segment_df[month_col] == month]
                    if len(actual_segment) == 0:
                        continue

                    actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)

                    # Count distinct accounts for segment
                    base_segment_count = train_segment[account_id_col].nunique()
                    actual_segment_count = actual_segment[account_id_col].nunique()

                    # Get all bins
                    all_bins = train_baseline.index.union(actual_dist.index)

                    for bin_name in all_bins:
                        # Simplified epsilon logic - no redundancy
                        expected_pct = train_baseline.get(bin_name, 0)
                        actual_pct = actual_dist.get(bin_name, 0)

                        # Add epsilon only if zero
                        expected_pct = epsilon if expected_pct == 0 else expected_pct
                        actual_pct = epsilon if actual_pct == 0 else actual_pct

                        # Calculate bin-level PSI
                        bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct)

                        # Get bin range information
                        bin_ranges = binning_info[feature]['bin_ranges']
                        if bin_name in bin_ranges:
                            bin_min = bin_ranges[bin_name]['min']
                            bin_max = bin_ranges[bin_name]['max']
                            bin_range = bin_ranges[bin_name]['range_str']
                        else:
                            # For categorical or special bins (Missing, Others)
                            bin_min = None
                            bin_max = None
                            bin_range = bin_name

                        results.append({
                            'Feature': feature,
                            'Feature_Type': binning_info[feature]['type'],
                            'Segment_Column': segment_col,
                            'Segment_Value': segment_val,
                            'Month': f"{month}",
                            'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                            'Current_Month': month,
                            'Base_Count': base_segment_count,
                            'Actual_Count': actual_segment_count,
                            'Bin': bin_name,
                            'Bin_Range': bin_range,
                            'Bin_Min': bin_min,
                            'Bin_Max': bin_max,
                            'Base_Percentage': (train_baseline.get(bin_name, 0) * 100),
                            'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
                            'Bin_PSI': bin_psi
                        })

    return pd.DataFrame(results)

## SIL

#### Query from risk_mart.sil_risk_ds_master_20230101_20250309_v2

##### 'Alpha - CIC-SIL-Model'

In [8]:
sq = """select distinct
    r.customerId customer_id ,
    r.digitalLoanAccountId,
    r.cic_score,
    r.cic_Personal_Loans_granted_contracts_amt_24M,
    r.cic_days_since_last_inquiry, 
    r.cic_cnt_active_contracts,
    r.cic_vel_contract_nongranted_cnt_12on24,
    r.cic_max_amt_granted_24M, 
    r.cic_zero_non_granted_ever_flag,
    r.cic_tot_active_contracts_util,
    r.cic_vel_contract_granted_amt_12on24,
    r.cic_zero_granted_ever_flag,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from risk_mart.sil_risk_ds_master_20230101_20250309_v2 r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId

where cic_score is not null;"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID e95b97d2-fab0-41c9-96e4-980f8b4fe9f4 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (204453, 13)


In [9]:
feature_column = ['cic_Personal_Loans_granted_contracts_amt_24M',
       'cic_days_since_last_inquiry', 'cic_cnt_active_contracts',
       'cic_vel_contract_nongranted_cnt_12on24', 'cic_max_amt_granted_24M',
       'cic_zero_non_granted_ever_flag', 'cic_tot_active_contracts_util',
       'cic_vel_contract_granted_amt_12on24', 'cic_zero_granted_ever_flag']

In [10]:
dfd = transform_data(data, feature_column, a='cic_score', modelDisplayName='Alpha - CIC-SIL-Model') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2591981,0f71ab25-0be6-4237-8d8e-4f166ee49169,f8f3923d-f0c8-4779-a8e8-448404077fe3,0.126862,2025-11-28T13:42:43.351477,2025-11-28T13:42:43.351477,Alpha - CIC-SIL-Model,v1,"{""cic_days_since_last_inquiry"": 1825.0, ""cic_z...",sil_march 25 models,19ab0940-aadf-4992-a0a2-d51e28c8479b,2025-11-28T13:42:43.351477,{},,android
1,2482970,3f9365df-7375-4c14-a28b-86a26b423a84,df367e25-ec50-45dd-8add-2d6f82cb114d,0.098358,2025-11-28T13:42:43.351477,2025-11-28T13:42:43.351477,Alpha - CIC-SIL-Model,v1,"{""cic_days_since_last_inquiry"": 174.0, ""cic_cn...",sil_march 25 models,2fdaa201-a2c6-44bb-aae5-3984dd2332f0,2025-11-28T13:42:43.351477,{},,android
2,2817917,2196fe98-95a6-4bfb-856c-5342f6bccf8f,11f5246d-d353-459f-8840-b41de939c09f,0.124686,2025-11-28T13:42:43.351477,2025-11-28T13:42:43.351477,Alpha - CIC-SIL-Model,v1,"{""cic_days_since_last_inquiry"": 1183.0, ""cic_z...",sil_march 25 models,830e99fd-06e5-48f0-9460-61f028002deb,2025-11-28T13:42:43.351477,{},,android
3,3262358,a383b3dd-28e1-4607-848b-e1a57c07a91a,2640ce0d-a657-4f01-9ccd-c6d9348fec04,0.173223,2025-11-28T13:42:43.351477,2025-11-28T13:42:43.351477,Alpha - CIC-SIL-Model,v1,"{""cic_days_since_last_inquiry"": 67.0, ""cic_vel...",sil_march 25 models,4c3182a9-4e56-4baf-9bb5-e80046a8cc2f,2025-11-28T13:42:43.351477,{},,android
4,3063565,a9bc00d9-196f-4591-bab2-47dfbf1805e0,a13d235e-a001-45a4-bbcc-e921136bcf0f,0.085567,2025-11-28T13:42:43.352472,2025-11-28T13:42:43.352472,Alpha - CIC-SIL-Model,v1,"{""cic_Personal_Loans_granted_contracts_amt_24M...",sil_march 25 models,cf8dd11c-37c9-4bfb-a912-ac83515a2045,2025-11-28T13:42:43.352472,{},,android


In [11]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=768bb745-e1fa-4c2c-82d5-34f83a4f4f10>

##### Alpha Sil Stack Model 

In [12]:
sq = """ 
select distinct 
r.customerId customer_id ,
r.digitalLoanAccountId,
r.alpha_stack_score,
r.beta_demo_score,
r.cic_score,
r.apps_score,
r.credo_gen_score,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where alpha_stack_score is not null
;
"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 4dcabc22-395a-4e23-9a0c-29c7960da6c6 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (204453, 8)


In [13]:
feature_column = ['beta_demo_score',
       'cic_score', 'apps_score',
       'credo_gen_score']

dfd = transform_data(data, feature_column, a='alpha_stack_score', modelDisplayName='Alpha - StackingModel') 
dfd.head()


,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2424505,dc4ed5d8-69fb-4fb8-a132-8e07ead7ae93,24b2184c-136b-48c0-8de8-bb1a7b5fd2a7,0.059519,2025-11-28T13:43:14.780526,2025-11-28T13:43:14.780526,Alpha - StackingModel,v1,"{""beta_demo_score"": 0.06515353039530854, ""cic_...",sil_march 25 models,a29cd994-60b6-44db-8797-8d7a20d883b8,2025-11-28T13:43:14.780526,{},,android
1,3023906,d0af11a8-2367-478b-9c19-69cd70d54942,2070efcb-1eb5-4ffa-8adc-40265c20273b,0.055909,2025-11-28T13:43:14.781526,2025-11-28T13:43:14.781526,Alpha - StackingModel,v1,"{""beta_demo_score"": 0.0713566097707466, ""cic_s...",sil_march 25 models,d5fd7711-f06e-4c04-878c-e412fb7ee036,2025-11-28T13:43:14.781526,{},,android
2,2665520,f662dc19-187f-47e7-b07e-c288714060d4,febf5d94-eef6-4797-8d38-99046fd8049e,0.085580,2025-11-28T13:43:14.781526,2025-11-28T13:43:14.781526,Alpha - StackingModel,v1,"{""beta_demo_score"": 0.09625384281300788, ""cic_...",sil_march 25 models,dd6c6d9e-723e-467a-8084-97561519031d,2025-11-28T13:43:14.781526,{},,android
3,2817917,2196fe98-95a6-4bfb-856c-5342f6bccf8f,23dcfacc-b9b5-4740-80a6-52e6161c39af,0.257212,2025-11-28T13:43:14.781526,2025-11-28T13:43:14.781526,Alpha - StackingModel,v1,"{""beta_demo_score"": 0.1350818237118101, ""cic_s...",sil_march 25 models,546bc050-528b-43c5-972b-43bfe17a2fd9,2025-11-28T13:43:14.781526,{},,android
4,2348760,b97e6d9a-09b5-41e8-adce-0d03665873d5,17c47099-6a9c-427d-8513-f80515fa6758,0.122655,2025-11-28T13:43:14.781526,2025-11-28T13:43:14.781526,Alpha - StackingModel,v1,"{""beta_demo_score"": 0.14179810818976543, ""cic_...",sil_march 25 models,ac371ccd-2758-492b-94e1-254d83faac38,2025-11-28T13:43:14.781526,{},,android


In [14]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=a0c1ef3a-56db-4c85-9ca3-0c47cf6d9ea3>

##### Beta Sil App Score

In [15]:
sq = """ 
select distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.apps_score,
r.app_cnt_absence_tag_30d,
r.app_cnt_absence_tag_90d ,
r.app_cnt_business_ever ,
r.app_cnt_competitors_30d ,
r.app_cnt_competitors_90d ,
r.app_cnt_education_ever ,
r.app_cnt_finance_7d ,
r.app_cnt_finance_90d ,
r.app_cnt_music_and_audio_ever ,
r.app_cnt_payday_90d ,
r.app_cnt_rated_for_3plus_ever ,
r.app_cnt_travel_and_local_ever ,
r.app_first_competitors_install_to_apply_days ,
r.app_first_payday_install_to_apply_days ,
r.app_median_time_bw_installed_mins_30d ,
r.app_vel_finance_30_over_365 ,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where apps_score is not null
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 1664026d-d33a-4a4f-8579-9dc290308951 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (317385, 20)


In [16]:
feature_column = ['app_cnt_rated_for_3plus_ever',
       'app_cnt_education_ever', 'app_cnt_business_ever',
       'app_cnt_music_and_audio_ever',
       'app_cnt_travel_and_local_ever', 'app_cnt_finance_7d',
       'app_cnt_competitors_30d', 'app_cnt_absence_tag_30d',
        'app_cnt_absence_tag_90d',
       'app_cnt_finance_90d', 'app_cnt_competitors_90d',
       'app_cnt_payday_90d',
       'app_median_time_bw_installed_mins_30d',
       'app_first_competitors_install_to_apply_days',
       'app_first_payday_install_to_apply_days',
       'app_vel_finance_30_over_365']

dfd = transform_data(data, feature_column, a='apps_score', modelDisplayName='Beta - AppsScoreModel') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3265787,2fa5501c-fee8-49ce-b0fc-7598e7499d10,6ac08672-7e08-4d46-a53b-e8f665caf3a4,0.670536,2025-11-28T13:43:45.702007,2025-11-28T13:43:45.702007,Beta - AppsScoreModel,v1,"{""app_cnt_rated_for_3plus_ever"": 39.0, ""app_cn...",sil_march 25 models,3688962c-b5a1-4a89-9189-b976328d02a5,2025-11-28T13:43:45.702007,{},,android
1,3045258,a1f6d39b-915a-41f5-b5f8-969ae1d0412c,a69029d8-d42f-46e7-962c-a1ef7ff6be9d,0.533583,2025-11-28T13:43:45.702007,2025-11-28T13:43:45.702007,Beta - AppsScoreModel,v1,"{""app_cnt_rated_for_3plus_ever"": 38.0, ""app_cn...",sil_march 25 models,f6ce0581-dda0-440d-9af0-d986ce676bd4,2025-11-28T13:43:45.702007,{},,android
2,2272982,8d848bbe-ef4d-4aaf-bb07-cde9166afe01,685a5540-84b3-42c9-b988-b475e754ac13,0.717967,2025-11-28T13:43:45.703019,2025-11-28T13:43:45.703019,Beta - AppsScoreModel,v1,"{""app_cnt_rated_for_3plus_ever"": 29.0, ""app_cn...",sil_march 25 models,2423faaf-4e32-434a-9939-2b9df8cd72a5,2025-11-28T13:43:45.703019,{},,android
3,2069036,ee6276eb-66a3-47a3-a4e8-06a8425ce82a,2feed125-8cec-4059-adce-e1e82bca6714,0.675898,2025-11-28T13:43:45.703019,2025-11-28T13:43:45.703019,Beta - AppsScoreModel,v1,"{""app_cnt_rated_for_3plus_ever"": 54.0, ""app_cn...",sil_march 25 models,4b28e70e-254b-46b6-aeee-a996eaf8c934,2025-11-28T13:43:45.703019,{},,android
4,3123422,9d297e5e-014b-458d-ba05-593c1c7e62ac,afd476db-782e-4c52-8ff4-7dcbf9caaad0,0.341867,2025-11-28T13:43:45.703019,2025-11-28T13:43:45.703019,Beta - AppsScoreModel,v1,"{""app_cnt_rated_for_3plus_ever"": 62.0, ""app_cn...",sil_march 25 models,2ade5db1-a62a-4331-9293-82b43189b6e8,2025-11-28T13:43:45.703019,{},,android


In [17]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=a6fce14d-1274-45b3-b93e-e4ba3ba08e29>

##### Beta SIL Demo Score

In [18]:
sq = """
select distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.beta_demo_score,
r.beta_de_ln_vas_opted_flag ,
r.beta_de_ln_doc_type_rolled ,
r.beta_de_ln_marital_status ,
r.beta_de_ln_age_bin ,
r.beta_de_ln_province_bin ,
r.beta_de_ln_ref2_type ,
r.beta_de_ln_education_level ,
r.beta_de_ln_ref1_type ,
r.beta_de_ln_industry_new_bin ,
r.beta_de_ln_appln_day_of_week ,
r.beta_de_onb_name_email_match_score ,
r.beta_de_ln_employment_type_new_bin ,
r.beta_de_ln_telconame ,
r.beta_de_time_bw_onb_loan_appln_mins ,
r.beta_de_ln_source_of_funds_new_bin ,
r.beta_de_ln_brand_bin ,
r.beta_de_ln_email_primary_domain ,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where beta_demo_score is not null
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID a314916f-d0b4-49a9-97fc-c86e008812bf successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (349207, 21)


In [19]:
feature_column = ['beta_de_ln_vas_opted_flag',
       'beta_de_ln_doc_type_rolled', 'beta_de_ln_marital_status',
       'beta_de_ln_age_bin', 'beta_de_ln_province_bin',
       'beta_de_ln_ref2_type', 'beta_de_ln_education_level',
       'beta_de_ln_ref1_type', 'beta_de_ln_industry_new_bin',
       'beta_de_ln_appln_day_of_week',
       'beta_de_onb_name_email_match_score',
       'beta_de_ln_employment_type_new_bin', 'beta_de_ln_telconame',
       'beta_de_time_bw_onb_loan_appln_mins',
       'beta_de_ln_source_of_funds_new_bin', 'beta_de_ln_brand_bin',
       'beta_de_ln_email_primary_domain']

dfd = transform_data(data, feature_column, a='beta_demo_score', modelDisplayName='Beta - DemoScoreModel') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2900115,c452e67d-2678-4300-8542-c6173feeed86,0d30667f-8ade-4ea3-bde3-3cfeca25ac06,0.189395,2025-11-28T13:45:00.639810,2025-11-28T13:45:00.639810,Beta - DemoScoreModel,v1,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",sil_march 25 models,e6ca8b8e-f7e5-4b8d-a246-6693ce6f7893,2025-11-28T13:45:00.639810,{},,android
1,2492451,c9200c70-46ff-4c8c-ace2-ef7fae3f6be1,1f3b5fef-6e14-4a26-a011-46cf923c544a,0.088578,2025-11-28T13:45:00.639810,2025-11-28T13:45:00.639810,Beta - DemoScoreModel,v1,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",sil_march 25 models,1ea4a5c8-7870-45c4-8e45-7bfb546ca0a7,2025-11-28T13:45:00.639810,{},,android
2,1389078,aea89823-083b-4539-8c85-847bcd2a48a7,e2e2980d-fdae-4a97-b5b9-41898192e9df,0.292979,2025-11-28T13:45:00.639810,2025-11-28T13:45:00.639810,Beta - DemoScoreModel,v1,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",sil_march 25 models,4462d6bb-79d0-45de-bfb3-5db844723e5b,2025-11-28T13:45:00.639810,{},,ios
3,2260235,df17e0c7-ca22-4a85-903e-5b272921da7c,ad0ee727-c5d1-40a2-8ec0-93c535505f62,0.201821,2025-11-28T13:45:00.640809,2025-11-28T13:45:00.640809,Beta - DemoScoreModel,v1,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",sil_march 25 models,31195c18-74fe-4edf-b459-534755f9eb3e,2025-11-28T13:45:00.640809,{},,android
4,2086964,506a950d-f0b9-4c81-8f32-af86342d6d11,308ea91f-6bd9-47a5-8b5d-e20a8e796636,0.250010,2025-11-28T13:45:00.641146,2025-11-28T13:45:00.641146,Beta - DemoScoreModel,v1,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",sil_march 25 models,2733e933-5702-45e1-80ad-38bf2b8cd7ba,2025-11-28T13:45:00.641146,{},,android


In [20]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=c6156e77-8ebb-42bb-9825-3c3102618916>

##### Beta SIL STACK Score Model

In [21]:
sq = """ 
select  distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.beta_stack_score,
r.apps_score,
r.credo_gen_score,
r.beta_demo_score,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where beta_stack_score is not null
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 6f3bbab9-576f-41c8-9977-446a9238cea8 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (349207, 7)


In [22]:
feature_column = ['apps_score', 'credo_gen_score', 'beta_demo_score']
dfd = transform_data(data, feature_column, a='beta_stack_score', modelDisplayName='Beta - StackScoreModel') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2599660,d429009e-91a0-4ef2-a9ad-95a06c4d3f25,4176fea1-20cd-42aa-aede-d86259881fe8,0.041279,2025-11-28T13:46:35.226274,2025-11-28T13:46:35.226274,Beta - StackScoreModel,v1,"{""apps_score"": 0.4444840736872238, ""credo_gen_...",sil_march 25 models,4ca960be-c95c-4fe0-92fa-942c929f8772,2025-11-28T13:46:35.226274,{},,android
1,2482970,3f9365df-7375-4c14-a28b-86a26b423a84,70e9c3a5-9ec0-48de-a7f8-c5efe7c19786,0.012756,2025-11-28T13:46:35.226274,2025-11-28T13:46:35.226274,Beta - StackScoreModel,v1,"{""apps_score"": 0.2851733508388537, ""credo_gen_...",sil_march 25 models,feea6b61-150b-4532-af5e-e2c99966f2ff,2025-11-28T13:46:35.226274,{},,android
2,3085867,d290345d-9405-4ef2-a116-bfb24f0ff572,39e57027-b944-4c35-bce9-ba894ba979ad,0.239083,2025-11-28T13:46:35.226274,2025-11-28T13:46:35.226274,Beta - StackScoreModel,v1,"{""apps_score"": 0.6561141858172787, ""credo_gen_...",sil_march 25 models,6f7d9ac0-9b49-427a-9aeb-5500699fe162,2025-11-28T13:46:35.226274,{},,android
3,2942780,7ab9eaaa-bad1-4a7c-8ddf-420f67e6e29c,b2e31691-2aa8-4f4c-909d-26ace2b04a63,0.351771,2025-11-28T13:46:35.226274,2025-11-28T13:46:35.226274,Beta - StackScoreModel,v1,"{""apps_score"": 0.6786980573874595, ""credo_gen_...",sil_march 25 models,4a1caeb7-2471-4ee5-a06e-1ade8fa8bc85,2025-11-28T13:46:35.226274,{},,android
4,2475587,50030cb7-1097-4460-829b-8105abbf4bb6,1106044e-ecea-44cf-abb7-0222f3c43d16,0.176778,2025-11-28T13:46:35.226274,2025-11-28T13:46:35.226274,Beta - StackScoreModel,v1,"{""apps_score"": 0.5759108325178293, ""credo_gen_...",sil_march 25 models,45aff8ad-d942-4bd6-a0ed-ca198b9ceb49,2025-11-28T13:46:35.226274,{},,android


In [23]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=45934ea9-6f7c-44d8-b448-d40b555e92ed>

##### Alpha  - IncomeEstimationModel

In [24]:
sq = """  
Select 
distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.alpha_estimated_income,
r.inc_alpha_cic_credit_avg_credit_limit,
r.inc_alpha_cic_max_active_contracts_amt,
r.inc_alpha_ln_company_name,
r.inc_alpha_ln_age,
r.inc_alpha_doc_type_rolled,
r.inc_alpha_ln_brand,
r.inc_alpha_ln_city,
r.inc_alpha_ln_cnt_dependents,
r.inc_alpha_ln_education_level,
r.inc_alpha_ln_employment_type_new,
r.inc_alpha_ln_gender,
r.inc_alpha_ln_industry_new,
r.inc_alpha_ln_loan_prod_type,
r.inc_alpha_ln_marital_status_new,
r.inc_alpha_ln_nature_of_work_new,
r.inc_alpha_ln_osversion_bin,
r.inc_alpha_ln_purpose,
r.inc_alpha_ln_source_of_funds_new,
r.inc_alpha_loan_monthly_income,
r.inc_alpha_encoded_company_name_grouped,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where r.alpha_estimated_income is not null
;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID 2f39a14e-d8b4-4805-9e72-5549b2587aa0 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (349207, 24)


In [25]:
feature_column = ['inc_alpha_cic_credit_avg_credit_limit',
       'inc_alpha_cic_max_active_contracts_amt', 'inc_alpha_ln_age',
       'inc_alpha_doc_type_rolled', 'inc_alpha_ln_brand', 'inc_alpha_ln_city',
       'inc_alpha_ln_cnt_dependents', 'inc_alpha_ln_education_level',
       'inc_alpha_ln_employment_type_new', 'inc_alpha_ln_gender',
       'inc_alpha_ln_industry_new', 'inc_alpha_ln_loan_prod_type',
       'inc_alpha_ln_marital_status_new', 'inc_alpha_ln_nature_of_work_new',
       'inc_alpha_ln_osversion_bin', 'inc_alpha_ln_purpose',
       'inc_alpha_ln_source_of_funds_new',
       'inc_alpha_encoded_company_name_grouped']
dfd = transform_data(data, feature_column, a='alpha_estimated_income', modelDisplayName='Alpha  - IncomeEstimationModel') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2092854,c6cb7231-44a8-4d79-a55b-5bc801dc176a,cedabb77-08be-4fcb-9633-c7e6cd1881cb,8875.825400,2025-11-28T13:47:21.898903,2025-11-28T13:47:21.898903,Alpha - IncomeEstimationModel,v1,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",sil_march 25 models,d9ed4285-3cd0-409f-872b-299e6e56e2dc,2025-11-28T13:47:21.898903,{},,ios
1,3022123,c80e6f27-fd77-48aa-ba6e-3dd110b82a89,e3516b78-d273-4fab-9e50-a2a346d770af,8281.401908,2025-11-28T13:47:21.898903,2025-11-28T13:47:21.898903,Alpha - IncomeEstimationModel,v1,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",sil_march 25 models,a5950074-ed67-48d1-877e-ccc8b2c3c9ee,2025-11-28T13:47:21.898903,{},,android
2,2201123,5ea8db47-82b2-4d7a-83e2-f60d9a8d5dee,e2dc9636-fc01-44bb-a051-8a07ca0c1648,9026.302206,2025-11-28T13:47:21.899905,2025-11-28T13:47:21.899905,Alpha - IncomeEstimationModel,v1,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",sil_march 25 models,7737b8f5-ab18-4a8b-a791-1020028846f7,2025-11-28T13:47:21.899905,{},,android
3,3088549,5d9adfd7-dafb-44e8-aaf2-c3bf1e7561b5,e202f739-ec8d-4450-8ba3-ba2294052482,9007.570939,2025-11-28T13:47:21.899905,2025-11-28T13:47:21.899905,Alpha - IncomeEstimationModel,v1,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",sil_march 25 models,ba0d7f01-6c0e-4825-b1c5-180e6956d30d,2025-11-28T13:47:21.899905,{},,android
4,2406932,13acac8f-8778-414b-9a3d-6725afea3976,0199d9e6-cf3b-4577-861c-3cc1a676c289,8824.198755,2025-11-28T13:47:21.899905,2025-11-28T13:47:21.899905,Alpha - IncomeEstimationModel,v1,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",sil_march 25 models,97eab50c-3dd8-4343-9613-b5ca162c89ad,2025-11-28T13:47:21.899905,{},,android


In [26]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=2ba22ff3-80fe-4c2e-a743-9a384362c9b1>

##### Beta - IncomeEstimationModelm

In [27]:
sq = """ 
select
distinct
r.customerId customer_id ,
r.digitalLoanAccountId,
r.beta_estimated_income,
r.inc_beta_ln_loan_type,
r.inc_beta_ln_education_level,
r.inc_beta_ln_employment_type_new,
r.inc_beta_ln_industry_new,
r.inc_beta_ln_age,
r.inc_beta_ln_brand,
r.inc_beta_ln_city,
r.inc_beta_ln_purpose,
r.inc_beta_ln_osversion_bin,
r.inc_beta_ln_postal_code,
r.inc_beta_ln_gender,
r.inc_beta_ln_doc_type_rolled,
r.inc_beta_ln_cnt_dependents,
r.inc_beta_ln_source_of_funds_new,
r.inc_beta_ln_marital_status_new,
r.inc_beta_encoded_company_name_grouped,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from `risk_mart.sil_risk_ds_master_20230101_20250309_v2` r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
where r.alpha_estimated_income is not null
;
"""

# data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
data = client.query(sq).result().to_arrow().to_pandas()
print(f"The shape of the dataframe is:\t {data.shape}")

The shape of the dataframe is:	 (349207, 20)


In [28]:
feature_column = ['inc_beta_ln_loan_type',
       'inc_beta_ln_education_level', 'inc_beta_ln_employment_type_new',
       'inc_beta_ln_industry_new', 'inc_beta_ln_age', 'inc_beta_ln_brand',
       'inc_beta_ln_city', 'inc_beta_ln_purpose', 'inc_beta_ln_osversion_bin',
       'inc_beta_ln_postal_code', 'inc_beta_ln_gender',
       'inc_beta_ln_doc_type_rolled', 'inc_beta_ln_cnt_dependents',
       'inc_beta_ln_source_of_funds_new', 'inc_beta_ln_marital_status_new',
       'inc_beta_encoded_company_name_grouped',]

dfd = transform_data(data, feature_column, a='beta_estimated_income', modelDisplayName='Beta - IncomeEstimationModel') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3216824,d316fffe-6a68-4ab5-bc98-9ad9e7fab5f4,87b13548-d770-4992-867a-f7a03126d819,23682.979623,2025-11-28T13:48:45.173927,2025-11-28T13:48:45.173927,Beta - IncomeEstimationModel,v1,"{""inc_beta_ln_loan_type"": ""SIL ZERO"", ""inc_bet...",sil_march 25 models,826f7b1a-d040-4bc4-8d8d-0a2de168c8ca,2025-11-28T13:48:45.173927,{},,android
1,2900074,999f192c-78e4-4a5a-a321-c6d8f15f17cf,6d835dc7-f8da-419f-8b56-b4cc15762dbe,7985.078336,2025-11-28T13:48:45.173927,2025-11-28T13:48:45.173927,Beta - IncomeEstimationModel,v1,"{""inc_beta_ln_loan_type"": ""SIL-Instore"", ""inc_...",sil_march 25 models,7336877a-5110-4104-9426-163e09b877bd,2025-11-28T13:48:45.173927,{},,ios
2,3069236,b6309686-6b9f-4b86-9be5-41edf4a2310c,180fa8b3-0256-4545-91b8-5c21552adb8c,8705.213847,2025-11-28T13:48:45.173927,2025-11-28T13:48:45.173927,Beta - IncomeEstimationModel,v1,"{""inc_beta_ln_loan_type"": ""SIL-Instore"", ""inc_...",sil_march 25 models,7983337a-4488-45a6-9645-f4bb8b465ac6,2025-11-28T13:48:45.173927,{},,android
3,1875597,1f9b0d01-3883-4ebe-808e-ea40e81c6267,c56be545-a60b-4052-bb0e-6716fc6aa7f2,8372.736250,2025-11-28T13:48:45.174950,2025-11-28T13:48:45.174950,Beta - IncomeEstimationModel,v1,"{""inc_beta_ln_loan_type"": ""SIL-Instore"", ""inc_...",sil_march 25 models,2bc93274-6fdf-44a3-9a7b-b0760500f49f,2025-11-28T13:48:45.174950,{},,android
4,2337180,5e391a4d-8a68-49e9-97b9-8eb16121e67e,70aacb84-7c1c-4b39-b490-de9e5a26bbe8,8200.212610,2025-11-28T13:48:45.174950,2025-11-28T13:48:45.174950,Beta - IncomeEstimationModel,v1,"{""inc_beta_ln_loan_type"": ""SIL-Instore"", ""inc_...",sil_march 25 models,8113e85f-c05b-4c28-9d5a-0947d2545cc5,2025-11-28T13:48:45.174950,{},,android


In [29]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=39a3c078-6a1e-4f93-a6a7-34418370e2c7>

## Cash

##### Alpha Cash Stack Model

###### Trench 1

In [30]:
sq = """ 
select 
  r.customer_id,
  r.digitalLoanAccountId, 
  r.stack_score,
  r.demo_score,
  r.apps_score,
  r.credo_score,
  r.cic_score,
  r.stack_score_norm,
  r.ln_os_type osType,
from worktable_data_analysis.cash_alpha_trench1_applied_loans_backscored_20241001_20250930 r;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID a3a78ebf-df45-4e71-bdf2-f6dc23db6ebf successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (67807, 9)


In [31]:
feature_column = ['demo_score', 'apps_score', 'credo_score', 'cic_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='stack_score', modelDisplayName='Alpha-Cash-Stack-Model', tc='Trench 1', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3539915,04faa4fc-40df-4c35-8fd4-7873f4f27d7a,77ce99ba-a799-41b4-b1aa-c799d437a335,0.669111,2025-11-28T13:50:25.177967,2025-11-28T13:50:25.177967,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.5744555950383494, ""credo_scor...",Cash September 25 Models,ed8112c4-ab1c-4f7b-a2e7-f8c038d35e34,2025-11-28T13:50:25.177967,{},Trench 1,iOS
1,3556922,e30aad68-3baa-4439-a5cc-8a742648b200,0de6c7f2-1d0a-492b-b269-a63acb0d0fcf,0.316267,2025-11-28T13:50:25.177967,2025-11-28T13:50:25.177967,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.44932706141455475, ""credo_sco...",Cash September 25 Models,c0d51e26-dabe-40ee-a08f-2a19df80c378,2025-11-28T13:50:25.177967,{},Trench 1,iOS
2,3242405,16e5ac4b-e966-4212-9f7b-9d24bb15a8dc,78e66661-3fb8-4db6-8de3-c9726ed47f0e,0.311066,2025-11-28T13:50:25.177967,2025-11-28T13:50:25.177967,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.40506523274601347, ""credo_sco...",Cash September 25 Models,ad59f8af-deb9-4e06-b18c-3981eaa04e6f,2025-11-28T13:50:25.177967,{},Trench 1,iOS
3,3646632,30b64b4a-5982-4e8c-b96e-e064aa605845,47ad565c-f0e1-49e8-b2f5-24e7ac452a0f,0.533312,2025-11-28T13:50:25.178966,2025-11-28T13:50:25.178966,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.5261059729214972, ""credo_scor...",Cash September 25 Models,5e893e9e-49de-4ce5-9cdc-a867428ab637,2025-11-28T13:50:25.178966,{},Trench 1,iOS
4,3454325,c96d5134-cf15-4393-a867-2220c7f176cc,47320728-5ff3-4599-8462-79633fc38dd3,0.433226,2025-11-28T13:50:25.178966,2025-11-28T13:50:25.178966,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.39591531683005676, ""credo_sco...",Cash September 25 Models,8d19ae71-9fe1-4076-8aa3-7149e951cbb2,2025-11-28T13:50:25.178966,{},Trench 1,iOS


In [32]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=c0e450c2-0e44-407b-8e48-ad91ad8335b9>

###### Trench 2

In [33]:
sq = """ 
select 
  r.customer_id,
  r.digitalLoanAccountId, 
  r.stack_score,
  r.demo_score,
  r.apps_score,
  r.credo_score,
  r.cic_score,
  r.stack_score_norm,
  r.ln_os_type osType,
from worktable_data_analysis.cash_alpha_trench2_applied_loans_backscored_20241001_20250930 r;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 006e89c4-dce6-46a0-b811-97ac830f68ee successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (42982, 9)


In [34]:
feature_column = ['demo_score', 'apps_score', 'credo_score', 'cic_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='stack_score', modelDisplayName='Alpha-Cash-Stack-Model', tc='Trench 2', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3239336,9386c275-dbf0-4c0a-8d02-8071f592b37d,4e3f2ed7-c20c-4430-9031-0e3c4304b8e3,0.672123,2025-11-28T13:50:38.435412,2025-11-28T13:50:38.435412,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.6247828133286182, ""credo_scor...",Cash September 25 Models,f65ad820-080e-41ce-87e6-db09542edd9e,2025-11-28T13:50:38.435412,{},Trench 2,iOS
1,2234712,d5b9351b-4d02-4052-8296-8248cce581fe,463284e6-8fd4-4940-9a65-45c5461f0954,0.618780,2025-11-28T13:50:38.436419,2025-11-28T13:50:38.436419,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.4714931796785411, ""credo_scor...",Cash September 25 Models,436f98e7-f915-4f96-b460-5f7636270d68,2025-11-28T13:50:38.436419,{},Trench 2,iOS
2,1147827,d5fd2224-f704-414f-8294-4fc98451fa61,ce2c89c7-7c2d-48ec-a0d5-fcc9d15aab92,0.513876,2025-11-28T13:50:38.436419,2025-11-28T13:50:38.436419,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.5756542624288734, ""credo_scor...",Cash September 25 Models,1dcd5151-b67f-4bbc-83b1-f5b119bf8fca,2025-11-28T13:50:38.436419,{},Trench 2,iOS
3,1924318,db06a8d7-5eaa-42f0-8bca-c2709a7bfe39,7b408710-f26e-45f0-9203-0c88a32b019b,0.687939,2025-11-28T13:50:38.436419,2025-11-28T13:50:38.436419,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.575024075027459, ""credo_score...",Cash September 25 Models,a370ad09-90a5-4151-9ff9-c23a189bdc2a,2025-11-28T13:50:38.436419,{},Trench 2,iOS
4,3042999,517586f3-eb4c-4401-9ccf-f12da8d552fc,f2920b54-5765-49a3-bd28-a2da88e46520,0.585185,2025-11-28T13:50:38.436419,2025-11-28T13:50:38.436419,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.646694460802266, ""credo_score...",Cash September 25 Models,c18419ce-d17c-4f0a-bb55-a29542a8e3f0,2025-11-28T13:50:38.436419,{},Trench 2,iOS


In [35]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=be7620cb-5bfa-448d-aba8-ac1e56878045>

###### Trench 3

In [36]:
sq = """ 
select 
  r.customer_id,
  r.digitalLoanAccountId, 
  r.stack_score,
  r.demo_score,
  r.apps_score,
  r.credo_score,
  r.cic_score,
  r.stack_score_norm,
  r.ln_os_type osType,
from worktable_data_analysis.cash_alpha_trench3_applied_loans_backscored_20241001_20250930 r;
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID d0ca592b-d59c-4fad-b72e-4f3313c7531e successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (18269, 9)


In [37]:
feature_column = ['demo_score', 'apps_score', 'credo_score', 'cic_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='stack_score', modelDisplayName='Alpha-Cash-Stack-Model', tc='Trench 3', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3169262,96426bd5-17d6-4fe0-bd14-3d7aa84dbe34,595fb268-064c-422d-9b0e-2b6056bf5786,0.523043,2025-11-28T13:50:47.803493,2025-11-28T13:50:47.803493,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.424909388603803, ""credo_score...",Cash September 25 Models,aba03ff7-a450-4397-99df-2a84f731a40a,2025-11-28T13:50:47.803493,{},Trench 3,iOS
1,1494932,4c192f35-a31e-438e-8fd0-ec7273a5ae39,b22fa1ef-5803-4c18-927a-8a269e64e8c7,0.415618,2025-11-28T13:50:47.803493,2025-11-28T13:50:47.803493,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.3984364213949569, ""credo_scor...",Cash September 25 Models,1830b1df-592f-493e-9300-6385ac83514e,2025-11-28T13:50:47.803493,{},Trench 3,iOS
2,2735400,fb47df8c-0a1a-45c1-a10e-d2b53b9e2453,5163cbab-1596-4530-9ab3-248081d9bc19,0.373629,2025-11-28T13:50:47.803493,2025-11-28T13:50:47.803493,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.21634670864737435, ""credo_sco...",Cash September 25 Models,fc636cc4-0e43-450c-a477-b6dad4e02e04,2025-11-28T13:50:47.803493,{},Trench 3,iOS
3,1331628,ff48b3ad-9111-4ffe-9a58-4e838afc0410,56d8d46c-55e9-48f7-8d0d-763cc1c77dd2,0.400812,2025-11-28T13:50:47.803493,2025-11-28T13:50:47.803493,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.49639525992168604, ""credo_sco...",Cash September 25 Models,7c2b6963-9dd8-47a1-a6e2-f5f19185e86c,2025-11-28T13:50:47.803493,{},Trench 3,iOS
4,2225608,485aac6c-1684-4698-be83-c08fdb74a1fd,c87f9a57-ac60-4010-a603-3300dcdb90f1,0.397827,2025-11-28T13:50:47.803493,2025-11-28T13:50:47.803493,Alpha-Cash-Stack-Model,v1,"{""demo_score"": 0.5367553576956362, ""credo_scor...",Cash September 25 Models,a152a8f4-6f34-4733-b1b7-3787562441da,2025-11-28T13:50:47.803493,{},Trench 3,iOS


In [38]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=38c3181b-3080-4605-bb8a-0fc8d1371fdd>

##### Alpha Cash CIC Model

###### Trench 1

In [39]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,
r.ca_cic_score,
r.max_age_all_contracts_snapshot,
r.ratio_overdue_contracts_to_granted_contracts,
r.ScoreRange,
r.ln_loan_level_user_type,
r.has_ever_been_overdue,
r.latest_granted_contract_overdue_flag,
r.ratio_closed_over_new_granted_cnt_24M,
r.ratio_risky_contracts_to_granted_contracts,
r.Short_and_Term_Loans_granted_contracts_cnt_24M,
r.flg_zero_non_granted_ever,
r.Personal_Loans_granted_contracts_amt_24M,
r.CreditAvgCreditLimit,
r.flg_zero_granted_ever,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from worktable_data_analysis.cash_alpha_cic_all_applied_backscored_20240901_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 1'
;
"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID 6706fe27-e722-41d7-84aa-f61f2e3eeb02 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (77967, 17)


In [40]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'ca_cic_score',
       'max_age_all_contracts_snapshot',
       'ratio_overdue_contracts_to_granted_contracts', 'ScoreRange',
       'ln_loan_level_user_type', 'has_ever_been_overdue',
       'latest_granted_contract_overdue_flag',
       'ratio_closed_over_new_granted_cnt_24M',
       'ratio_risky_contracts_to_granted_contracts',
       'Short_and_Term_Loans_granted_contracts_cnt_24M',
       'flg_zero_non_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'CreditAvgCreditLimit', 'flg_zero_granted_ever', 'osType'],
      dtype='object')

In [41]:
feature_column = ['max_age_all_contracts_snapshot',
       'ratio_overdue_contracts_to_granted_contracts', 'ScoreRange',
       'ln_loan_level_user_type', 'has_ever_been_overdue',
       'latest_granted_contract_overdue_flag',
       'ratio_closed_over_new_granted_cnt_24M',
       'ratio_risky_contracts_to_granted_contracts',
       'Short_and_Term_Loans_granted_contracts_cnt_24M',
       'flg_zero_non_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'CreditAvgCreditLimit', 'flg_zero_granted_ever', 'ca_cic_score']

dfd = transform_data(data, feature_column, a='ca_cic_score', modelDisplayName='Alpha-Cash-CIC-Model', tc='Trench 1', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3094602,9858ef46-e926-42b2-8d5b-0af216b4df65,ddb89822-3555-49f7-8ecc-1f5f7bced5ca,0.417177,2025-11-28T13:50:55.687778,2025-11-28T13:50:55.687778,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 1825.0, ""ra...",Cash September 25 Models,948a2de0-c2fc-4d84-ac7c-3b6a67e7ede7,2025-11-28T13:50:55.687778,{},Trench 1,android
1,3491438,501b8477-330d-4624-9a12-66e681e99219,dfcd9c80-f0b5-41bf-a3bd-ae1ff9606a5d,0.636357,2025-11-28T13:50:55.688322,2025-11-28T13:50:55.688322,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 385.0, ""rat...",Cash September 25 Models,6a23b0e3-7c4a-4109-8710-4b81a9f28bec,2025-11-28T13:50:55.688322,{},Trench 1,ios
2,3677488,0a130c8e-8d65-4531-b96e-f1ae4ef5aa85,563f841e-6c99-480d-a3db-bad98e40df59,0.439237,2025-11-28T13:50:55.688322,2025-11-28T13:50:55.688322,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 1051.0, ""ra...",Cash September 25 Models,9ca8ae37-0426-4bd8-be52-c3f37dc90c65,2025-11-28T13:50:55.688322,{},Trench 1,ios
3,3241892,267b4dab-1970-4d8b-a513-f109ca0eb2b0,c8e2151f-7796-47a0-b4e7-7369dcd77a93,0.577894,2025-11-28T13:50:55.688322,2025-11-28T13:50:55.688322,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 158.0, ""rat...",Cash September 25 Models,5c2d2420-19a5-4c76-9631-72e3dd2c5312,2025-11-28T13:50:55.688322,{},Trench 1,android
4,3356846,ab50d9ad-a541-45e8-ae12-40b713c40693,0841c8d1-f1bd-4c0a-8160-cfcd9b486b17,0.527646,2025-11-28T13:50:55.688868,2025-11-28T13:50:55.688868,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 1829.0, ""ra...",Cash September 25 Models,b3e42905-15d8-49fe-bc30-99bdfe70ff55,2025-11-28T13:50:55.688868,{},Trench 1,android


In [42]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=2d2afb85-ba20-4235-9246-d0fbb4fe59ed>

###### Trench 2

In [43]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,
r.ca_cic_score,
r.max_age_all_contracts_snapshot,
r.ratio_overdue_contracts_to_granted_contracts,
r.ScoreRange,
r.ln_loan_level_user_type,
r.has_ever_been_overdue,
r.latest_granted_contract_overdue_flag,
r.ratio_closed_over_new_granted_cnt_24M,
r.ratio_risky_contracts_to_granted_contracts,
r.Short_and_Term_Loans_granted_contracts_cnt_24M,
r.flg_zero_non_granted_ever,
r.Personal_Loans_granted_contracts_amt_24M,
r.CreditAvgCreditLimit,
r.flg_zero_granted_ever,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from worktable_data_analysis.cash_alpha_cic_all_applied_backscored_20240901_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 2'
;
"""


data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 636cf161-4209-41bb-9db4-e7370bd5660e successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (48479, 17)


In [44]:
feature_column = ['max_age_all_contracts_snapshot',
       'ratio_overdue_contracts_to_granted_contracts', 'ScoreRange',
       'ln_loan_level_user_type', 'has_ever_been_overdue',
       'latest_granted_contract_overdue_flag',
       'ratio_closed_over_new_granted_cnt_24M',
       'ratio_risky_contracts_to_granted_contracts',
       'Short_and_Term_Loans_granted_contracts_cnt_24M',
       'flg_zero_non_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'CreditAvgCreditLimit', 'flg_zero_granted_ever', 'ca_cic_score']

dfd = transform_data(data, feature_column, a='ca_cic_score', modelDisplayName='Alpha-Cash-CIC-Model', tc='Trench 2', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,1660892,15490ed1-ac04-4cab-8e59-d0c74c733ac8,d0967c53-6c7e-47f8-892c-e2df47a91f15,0.530054,2025-11-28T13:51:13.578661,2025-11-28T13:51:13.578661,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 807.0, ""rat...",Cash September 25 Models,e81466fa-983f-454d-8b63-afcc579c45dd,2025-11-28T13:51:13.578661,{},Trench 2,ios
1,1773980,a2df33fb-84ac-4922-af24-bba9c8482113,84825fc9-1a54-4ce3-93dd-7536e70e1e62,0.529406,2025-11-28T13:51:13.578661,2025-11-28T13:51:13.578661,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 2102.0, ""ra...",Cash September 25 Models,b5ec2c79-15e7-448a-bee8-920089c6e81b,2025-11-28T13:51:13.578661,{},Trench 2,android
2,3081780,3f06184f-ba68-424f-b891-0de99ce7a686,d19154f5-3255-4856-9d5b-8abdfdc8a330,0.577223,2025-11-28T13:51:13.578661,2025-11-28T13:51:13.578661,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 869.0, ""rat...",Cash September 25 Models,1e30de85-0373-4ca0-87ad-bac744f2c4da,2025-11-28T13:51:13.578661,{},Trench 2,ios
3,1741970,76e1428a-c2e2-4395-8a8f-84205f769a17,1602f25f-3afd-4287-965d-1cc671a541b5,0.436432,2025-11-28T13:51:13.578661,2025-11-28T13:51:13.578661,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 2579.0, ""ra...",Cash September 25 Models,841cd7f9-d15a-47c6-ab6f-42b3fee8f587,2025-11-28T13:51:13.578661,{},Trench 2,ios
4,2562321,9df623fb-8b09-4c84-890f-85f28f65a8a0,30032569-590d-4d3b-80b5-913ae3b34080,0.486221,2025-11-28T13:51:13.578661,2025-11-28T13:51:13.578661,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 1825.0, ""ra...",Cash September 25 Models,0fe4f376-1709-4a9d-ae1d-125c18f0cca0,2025-11-28T13:51:13.578661,{},Trench 2,android


In [45]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=b5059b92-ae38-4f05-b664-57999d32bc77>

###### Trench 3

In [46]:
sq = """ 
select 
lmt.customerId customer_id,
r.digitalLoanAccountId,
r.ca_cic_score,
r.max_age_all_contracts_snapshot,
r.ratio_overdue_contracts_to_granted_contracts,
r.ScoreRange,
r.ln_loan_level_user_type,
r.has_ever_been_overdue,
r.latest_granted_contract_overdue_flag,
r.ratio_closed_over_new_granted_cnt_24M,
r.ratio_risky_contracts_to_granted_contracts,
r.Short_and_Term_Loans_granted_contracts_cnt_24M,
r.flg_zero_non_granted_ever,
r.Personal_Loans_granted_contracts_amt_24M,
r.CreditAvgCreditLimit,
r.flg_zero_granted_ever,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from worktable_data_analysis.cash_alpha_cic_all_applied_backscored_20240901_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 3'
;
"""


data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID fad4f91c-ea54-42e1-b11e-ee5dfea0e579 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (19300, 17)


In [47]:
feature_column = ['max_age_all_contracts_snapshot',
       'ratio_overdue_contracts_to_granted_contracts', 'ScoreRange',
       'ln_loan_level_user_type', 'has_ever_been_overdue',
       'latest_granted_contract_overdue_flag',
       'ratio_closed_over_new_granted_cnt_24M',
       'ratio_risky_contracts_to_granted_contracts',
       'Short_and_Term_Loans_granted_contracts_cnt_24M',
       'flg_zero_non_granted_ever', 'Personal_Loans_granted_contracts_amt_24M',
       'CreditAvgCreditLimit', 'flg_zero_granted_ever', 'ca_cic_score']

dfd = transform_data(data, feature_column, a='ca_cic_score', modelDisplayName='Alpha-Cash-CIC-Model', tc='Trench 3', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,1905112,8beaefa6-f137-494e-a58e-9ae659eff349,0aeaeb7d-cc20-46eb-808d-7956cc816834,0.567259,2025-11-28T13:51:26.315593,2025-11-28T13:51:26.315593,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 2667.0, ""ra...",Cash September 25 Models,68816d75-0ea8-457c-a504-69e67cb97e3d,2025-11-28T13:51:26.315593,{},Trench 3,android
1,2225416,97d9affc-d4f9-4269-87fd-fb598f80e853,b68597ee-6be1-475a-898a-162499862286,0.316980,2025-11-28T13:51:26.316133,2025-11-28T13:51:26.316133,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 2424.0, ""ra...",Cash September 25 Models,1fa697a7-dca1-4384-8da8-88a76da148f9,2025-11-28T13:51:26.316133,{},Trench 3,android
2,2316986,2df91939-220e-4932-8c07-5e2c09929ef3,9dee480c-82dc-4e75-a7c4-2aede05ba705,0.449782,2025-11-28T13:51:26.316133,2025-11-28T13:51:26.316133,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 346.0, ""rat...",Cash September 25 Models,ef922692-d0b6-48c3-94fc-f8d2177a202f,2025-11-28T13:51:26.316133,{},Trench 3,ios
3,2980389,d65f4d12-d433-423d-95e8-9dd6bed378c0,b787fa56-aaee-4735-bf71-3306e0d82675,0.317131,2025-11-28T13:51:26.316133,2025-11-28T13:51:26.316133,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 4448.0, ""ra...",Cash September 25 Models,7692d518-3614-49de-86aa-6a2227cb9f2e,2025-11-28T13:51:26.316133,{},Trench 3,android
4,3038157,7ed8e1f9-ef37-4d39-be05-093a5544b22c,38d56527-3398-4c92-862d-6e5e3330fe25,0.411814,2025-11-28T13:51:26.316133,2025-11-28T13:51:26.316133,Alpha-Cash-CIC-Model,v1,"{""max_age_all_contracts_snapshot"": 892.0, ""rat...",Cash September 25 Models,59e09620-e267-4971-9bb6-89da2049072e,2025-11-28T13:51:26.316133,{},Trench 3,ios


In [48]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=3d90f06f-a005-4841-92ff-5abb3925e4aa>

##### Beta-Cash-Demo-Model

###### Trench 1

In [49]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId,
r.c_demo_score Beta_Cash_Demo_Score,
r.ln_vas_opted_flag, 
r.ln_self_dec_income, 
r.ln_age,
r.ln_source_funds_new_bin, 
r.ln_loan_level_user_type,
r.ln_industry_new_cat_bin, 
r.ln_marital_status,
r.ln_doc_type_rolled, 
r.ln_education_level,
r.ln_ref2_type, 
r.ln_email_primary_domain, 
r.ln_province_bin,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from worktable_data_analysis.cash_beta_demo_all_applied_backscored_20241001_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 1'
;

"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID f50ee34c-a42f-45d4-8862-3e9479c96482 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (324441, 16)


In [50]:
feature_column = ['ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age',
       'ln_source_funds_new_bin', 'ln_loan_level_user_type',
       'ln_industry_new_cat_bin',
       'ln_marital_status',
       'ln_doc_type_rolled',
       'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain',
       'ln_province_bin','Beta_Cash_Demo_Score']

dfd = transform_data(data, feature_column, a='Beta_Cash_Demo_Score', modelDisplayName='Beta-Cash-Demo-Model', tc='Trench 1', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3524202,f30a3598-78c7-4c08-8a3b-e3f17e223874,6be90912-30cf-425b-ab8d-21277cfcf2a2,0.392189,2025-11-28T13:51:41.038057,2025-11-28T13:51:41.038057,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,6181663a-a48c-41a7-ba44-0139bee49ac1,2025-11-28T13:51:41.038057,{},Trench 1,android
1,3506130,e023c895-cbf3-41ab-bb89-3f5d6537fc18,995c8028-e04a-4514-a779-1b8fc0b813db,0.308441,2025-11-28T13:51:41.038057,2025-11-28T13:51:41.038057,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,bb929556-176c-46dc-a08e-e7f8a2e6bca6,2025-11-28T13:51:41.038057,{},Trench 1,android
2,3685208,ef2d81ee-6c32-44df-aeb4-b430692fd1a7,3b72f146-ea76-45ca-b3b7-205dcc4f47da,0.336830,2025-11-28T13:51:41.038057,2025-11-28T13:51:41.038057,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,0430e299-ed68-4bac-9d69-478fe625d527,2025-11-28T13:51:41.038057,{},Trench 1,android
3,3473529,c671b1b3-ccaa-4ecf-93b6-67dc26a94aef,565cc83a-859b-443b-b97a-8a5448d00aea,0.236228,2025-11-28T13:51:41.038057,2025-11-28T13:51:41.038057,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,4a2de6f0-f2f6-4197-8072-010045e447bd,2025-11-28T13:51:41.038057,{},Trench 1,ios
4,3646085,4ec7cc47-49c2-469c-8a08-cbb980f6f147,1137acf6-dd0a-4c8f-9296-b97c7df6ae66,0.550242,2025-11-28T13:51:41.038057,2025-11-28T13:51:41.038057,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,3435bc82-9c64-4a6c-a400-849e80384140,2025-11-28T13:51:41.038057,{},Trench 1,ios


In [51]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=9efa7ce9-35aa-44f4-8a00-8c870f6b7b35>

###### Trench 2

In [52]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId,
r.c_demo_score Beta_Cash_Demo_Score,
r.ln_vas_opted_flag, 
r.ln_self_dec_income, 
r.ln_age,
r.ln_source_funds_new_bin, 
r.ln_loan_level_user_type,
r.ln_industry_new_cat_bin, 
r.ln_marital_status,
r.ln_doc_type_rolled, 
r.ln_education_level,
r.ln_ref2_type, 
r.ln_email_primary_domain, 
r.ln_province_bin,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from worktable_data_analysis.cash_beta_demo_all_applied_backscored_20241001_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 2'
;

"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID a8b5f016-2892-4417-b2cf-c47ad15a6f51 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (124539, 16)


In [53]:
feature_column = ['ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age',
       'ln_source_funds_new_bin', 'ln_loan_level_user_type',
       'ln_industry_new_cat_bin',
       'ln_marital_status',
       'ln_doc_type_rolled',
       'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain',
       'ln_province_bin','Beta_Cash_Demo_Score']

dfd = transform_data(data, feature_column, a='Beta_Cash_Demo_Score', modelDisplayName='Beta-Cash-Demo-Model', tc='Trench 2', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3109175,b2f82931-b097-47ea-a94f-d465a129e234,c3829a1a-03c5-4647-bd44-3cf50f7648b0,0.446606,2025-11-28T13:52:55.129106,2025-11-28T13:52:55.129106,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,28cd0745-d4e9-4869-bf9e-a5ab61da76d6,2025-11-28T13:52:55.129106,{},Trench 2,android
1,1989614,13736cde-4bfa-4b68-a2d6-c8c114ecfa12,26e338e5-4bc3-475e-809e-51be7ecba825,0.348240,2025-11-28T13:52:55.129106,2025-11-28T13:52:55.129106,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,eceab375-a1e9-4522-ba39-1525f57701b6,2025-11-28T13:52:55.129106,{},Trench 2,android
2,1158355,bd0af952-7415-4342-bfb2-14a343c8467b,d9a834c4-7e8c-43d1-8248-b540d7fd31ca,0.192801,2025-11-28T13:52:55.130106,2025-11-28T13:52:55.130106,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,00f21f3a-0ced-442d-9605-ac0ede1ded2b,2025-11-28T13:52:55.130106,{},Trench 2,android
3,2933617,ff508f99-beec-4c1f-b037-dbb667c7d106,967fdad5-1642-461d-a6e0-77275fc5d6a7,0.617165,2025-11-28T13:52:55.130106,2025-11-28T13:52:55.130106,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,01b822df-9518-4556-9f63-ca82f5f4de83,2025-11-28T13:52:55.130106,{},Trench 2,ios
4,1399464,6c455b29-2a98-4245-9133-b433379ae8dd,9b99f07f-f1d1-4e6e-962c-f16aae6bbb36,0.288250,2025-11-28T13:52:55.130106,2025-11-28T13:52:55.130106,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,763a358a-9e83-4194-ba6d-650e04b55671,2025-11-28T13:52:55.130106,{},Trench 2,ios


In [54]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=bf1ef249-21a6-43ec-8a96-7c6a62471464>

###### Trench 3

In [55]:
sq = """ 
select 
r.customerId customer_id,
r.digitalLoanAccountId,
r.c_demo_score Beta_Cash_Demo_Score,
r.ln_vas_opted_flag, 
r.ln_self_dec_income, 
r.ln_age,
r.ln_source_funds_new_bin, 
r.ln_loan_level_user_type,
r.ln_industry_new_cat_bin, 
r.ln_marital_status,
r.ln_doc_type_rolled, 
r.ln_education_level,
r.ln_ref2_type, 
r.ln_email_primary_domain, 
r.ln_province_bin,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
from worktable_data_analysis.cash_beta_demo_all_applied_backscored_20241001_20250930 r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where trench_category = 'Trench 3'
;

"""

data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 4ccf9a1a-4927-47f9-8603-1026f4f7ec95 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (42441, 16)


In [56]:
feature_column = ['ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age',
       'ln_source_funds_new_bin', 'ln_loan_level_user_type',
       'ln_industry_new_cat_bin',
       'ln_marital_status',
       'ln_doc_type_rolled',
       'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain',
       'ln_province_bin','Beta_Cash_Demo_Score']

dfd = transform_data(data, feature_column, a='Beta_Cash_Demo_Score', modelDisplayName='Beta-Cash-Demo-Model', tc='Trench 3', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2808659,90173e0a-a352-4046-bcdd-b859935786b7,92a4588a-f28a-4ce4-a7a3-4cecbc5a95b5,0.327727,2025-11-28T13:53:27.857024,2025-11-28T13:53:27.857024,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,9a28f4a2-e254-4dfb-b711-ebe2957cd735,2025-11-28T13:53:27.857024,{},Trench 3,android
1,3025558,dcf06e41-9faf-4239-93bd-a1d0af611967,f672b0d7-b5ae-4553-b36c-47d1d689a027,0.199038,2025-11-28T13:53:27.857547,2025-11-28T13:53:27.857547,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,b3db0d5a-9fa2-43f7-bdea-2308e084fa35,2025-11-28T13:53:27.857547,{},Trench 3,android
2,3222695,96e40cc5-1b5f-4f3b-a241-47133846d2aa,c608c6a7-905f-4bf6-add2-1de9539bbb00,0.376586,2025-11-28T13:53:27.858094,2025-11-28T13:53:27.858094,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",Cash September 25 Models,436b2184-3249-420b-b9b7-7b20a89d626f,2025-11-28T13:53:27.858094,{},Trench 3,android
3,2519686,2422803e-9f0a-4bd0-9b55-9574270bd5a5,8b0aa6c2-0a85-4851-8832-2eeaa2012a61,0.411020,2025-11-28T13:53:27.858094,2025-11-28T13:53:27.858094,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,ac665fd3-5f38-4bd7-9d2b-11c775cc9212,2025-11-28T13:53:27.858094,{},Trench 3,android
4,3133246,e01b65d0-b572-49b7-ad69-984953cf3db7,b0b117db-1b87-4e04-af9f-60d700eb4b3d,0.405028,2025-11-28T13:53:27.858094,2025-11-28T13:53:27.858094,Beta-Cash-Demo-Model,v1,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",Cash September 25 Models,d584c564-9b8a-4c09-92a8-df275f9cc7da,2025-11-28T13:53:27.858094,{},Trench 3,android


In [57]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=885c0eb1-85ad-49ec-873c-61b7007ee377>

##### Beta-Cash-Stack-Model

In [58]:
sq = """ 
select r.customer_id,
r.digitalLoanAccountId,
r.demo_score,
r.apps_score,
r.credo_score,
r.stack_score Beta_cash_stack_score,
r.stack_score_norm,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType
 from worktable_data_analysis.cash_beta_trench1_applied_loans_backscored_20241001_20250930 r
 left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 2979836d-d197-484e-8315-bb490e9947e9 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (324344, 8)


In [59]:
feature_column = ['demo_score',
       'apps_score', 'credo_score',
       'stack_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='Beta_cash_stack_score', modelDisplayName='Beta-Cash-Stack-Model', tc='Trench 1', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,3410687,1a5a3fd3-505a-46ef-b9c0-b3b135b99a3c,f444d725-43bd-494d-8656-adeb2cdb14c9,0.565596,2025-11-28T13:53:47.385022,2025-11-28T13:53:47.385022,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.5821279210364091, ""apps_score...",Cash September 25 Models,2fdadc79-ada8-4b12-b012-335017197118,2025-11-28T13:53:47.385022,{},Trench 1,android
1,3601097,8ff007f7-3ff6-493a-b1e4-ea9692aee75f,0a2bfb73-59c0-4fad-8b84-c28d56d0e162,0.550231,2025-11-28T13:53:47.385581,2025-11-28T13:53:47.385581,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.313943734123847, ""apps_score""...",Cash September 25 Models,5ea1a8db-d99d-4837-96ca-4dbb609fd24b,2025-11-28T13:53:47.385581,{},Trench 1,android
2,3637382,ae290c89-7b94-4cb2-bd0d-629330a0cf76,f224bafe-32ed-474f-a5f2-d2a6db06cddb,0.059553,2025-11-28T13:53:47.385581,2025-11-28T13:53:47.385581,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.34571255287671476, ""apps_scor...",Cash September 25 Models,10ad1b3c-1e62-41b5-8148-02afed81484e,2025-11-28T13:53:47.385581,{},Trench 1,android
3,2906013,8ec610cb-b414-474f-929d-63fe5c646d98,2026f4c8-fbe2-4c84-8e12-c7e445e28833,0.478854,2025-11-28T13:53:47.386121,2025-11-28T13:53:47.386121,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.5709528374115794, ""apps_score...",Cash September 25 Models,7fd3a6f6-f5c5-4809-89b4-5ce234943ecc,2025-11-28T13:53:47.386121,{},Trench 1,android
4,3239981,02f3ecb1-fa36-4c3d-8d51-b0b3dab850b0,8667d2b3-c528-4506-8adf-db075961d8a9,0.438490,2025-11-28T13:53:47.386121,2025-11-28T13:53:47.386121,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.4199113778756657, ""apps_score...",Cash September 25 Models,266c79f8-18ea-4da6-9bde-8b9aeaf4886d,2025-11-28T13:53:47.386121,{},Trench 1,android


In [60]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=86c794fb-5cde-4d56-a683-d9f4ffc1678d>

###### Trench 2

In [61]:
sq = """ 
select r.customer_id,
r.digitalLoanAccountId,
r.demo_score,
r.apps_score,
r.credo_score,
r.stack_score Beta_cash_stack_score,
r.stack_score_norm,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType
 from worktable_data_analysis.cash_beta_trench2_applied_loans_backscored_20241001_20250930 r
 left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 0a652c35-2d31-4754-bc20-f41f851340ad successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (121682, 8)


In [62]:
feature_column = ['demo_score',
       'apps_score', 'credo_score',
       'stack_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='Beta_cash_stack_score', modelDisplayName='Beta-Cash-Stack-Model', tc='Trench 2', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2910204,aa635cd9-df14-4e5f-b33c-5362c818a76b,3bd38a33-6680-4541-8beb-8de98e8eb317,0.462535,2025-11-28T13:54:44.854995,2025-11-28T13:54:44.854995,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.5589265112756996, ""apps_score...",Cash September 25 Models,bbe986e5-e368-4560-bc61-ce8653235ad4,2025-11-28T13:54:44.854995,{},Trench 2,android
1,3048818,071661c8-a744-4b70-b1d9-089e43ee9196,15242b23-1945-4706-8d3e-cf07e68d8d7e,0.612213,2025-11-28T13:54:44.854995,2025-11-28T13:54:44.854995,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.4194741600815599, ""apps_score...",Cash September 25 Models,8d5d6d38-be1a-4fae-8f4c-80283b1a6a6c,2025-11-28T13:54:44.854995,{},Trench 2,android
2,1514017,3f00d93a-fbda-4215-8d38-3efa5b02d87a,ae9873c4-23d8-412f-81dc-d1408cafde41,0.507886,2025-11-28T13:54:44.856335,2025-11-28T13:54:44.856335,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.39861781507934557, ""apps_scor...",Cash September 25 Models,b26a4fc8-30ec-4d7d-adfc-bdba1a65949e,2025-11-28T13:54:44.856335,{},Trench 2,android
3,3521505,3df99be3-b9aa-4147-9012-3e4724a8cf53,5f1f6db2-0b8e-44ce-8737-2e7c7d3d8f4f,0.544475,2025-11-28T13:54:44.856335,2025-11-28T13:54:44.856335,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.545336827591923, ""apps_score""...",Cash September 25 Models,0abcffd2-0f2f-4c15-86e0-593c95c53b10,2025-11-28T13:54:44.856335,{},Trench 2,android
4,3193266,7868e232-f839-4932-800d-252fc93ce57b,5adcd5d8-90dd-4767-95c3-8dfec90f558d,0.187083,2025-11-28T13:54:44.856335,2025-11-28T13:54:44.856335,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.5792912457612119, ""apps_score...",Cash September 25 Models,51778b6b-694d-4072-b9ac-e5d6f78f95af,2025-11-28T13:54:44.856335,{},Trench 2,android


In [63]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=d970cd77-a27c-4c31-8897-d3562a8ee6be>

###### Trench 3

In [64]:
sq = """ 
select r.customer_id,
r.digitalLoanAccountId,
r.demo_score,
r.apps_score,
r.credo_score,
r.stack_score Beta_cash_stack_score,
r.stack_score_norm,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType
 from worktable_data_analysis.cash_beta_trench3_applied_loans_backscored_20241001_20250930 r
 left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId;

"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")


Job ID 05587c23-a381-4cd9-85fc-c4b3427db02f successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (42414, 8)


In [65]:
feature_column = ['demo_score',
       'apps_score', 'credo_score',
       'stack_score', 'stack_score_norm']

dfd = transform_data(data, feature_column, a='Beta_cash_stack_score', modelDisplayName='Beta-Cash-Stack-Model', tc='Trench 3', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,1790980,25554d21-8c04-406a-a63b-a3c1bbf38920,42dc0777-ef8d-4613-ba53-79a198d4ca00,0.691787,2025-11-28T13:55:03.180416,2025-11-28T13:55:03.180416,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.528055106116327, ""apps_score""...",Cash September 25 Models,bd53cece-592a-4a03-bba8-32ac37502a0c,2025-11-28T13:55:03.180416,{},Trench 3,android
1,3301106,ce8424d1-5c41-455d-b907-e79c4be9b5cb,c6ea0458-c4bc-4f64-8fb9-c360d45e600e,0.445023,2025-11-28T13:55:03.180416,2025-11-28T13:55:03.180416,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.3306015341810304, ""apps_score...",Cash September 25 Models,11acaa3d-119c-46bc-805f-b79fc86271ef,2025-11-28T13:55:03.180416,{},Trench 3,android
2,2933897,31f32ff5-4c10-4b60-a669-bfc3c14eb787,c4df0e90-9fe1-47a7-8e6b-0339fe69dc05,0.568966,2025-11-28T13:55:03.180416,2025-11-28T13:55:03.180416,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.36685506509495747, ""apps_scor...",Cash September 25 Models,2a9ffbb2-2d4f-4a1c-bcdd-0d8a1de515b6,2025-11-28T13:55:03.180416,{},Trench 3,android
3,2652097,e8d3c4cf-984c-428c-a1e9-c0e9873fca4e,6d276305-d005-4eec-a37e-5aa6006eee59,0.792240,2025-11-28T13:55:03.180416,2025-11-28T13:55:03.180416,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.6272455507634074, ""apps_score...",Cash September 25 Models,7261ed1d-acef-482b-8e13-54c698337692,2025-11-28T13:55:03.180416,{},Trench 3,android
4,2478567,aa25d3ee-5f5e-410e-8efb-c35c612ed64b,a50ac466-5970-40ef-a852-cd8694aac4c7,0.793239,2025-11-28T13:55:03.180416,2025-11-28T13:55:03.180416,Beta-Cash-Stack-Model,v1,"{""demo_score"": 0.5768767421164935, ""apps_score...",Cash September 25 Models,79f460de-e4a8-403b-a074-941429e02f2d,2025-11-28T13:55:03.180416,{},Trench 3,android


In [66]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=cca854e4-722e-4929-8689-81b4cb5bb1f7>

#####  Beta Cash AppScore Model

###### Trench 1

In [67]:
## query to add the trench category trench 1 and trench 2

# create or replace table risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy as 
# select * from risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored;



# ALTER TABLE risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy
# ADD COLUMN trench_category STRING;

# MERGE risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy AS target
# USING (
#   SELECT 
#     a.digitalLoanAccountId,
#     CASE 
#       WHEN b.ln_loan_level_user_type = '2_New Applicant'
#            AND DATE_DIFF(DATE(b.ln_appln_submit_datetime), DATE(b.onb_tsa_onboarding_datetime), DAY) <= 30 THEN 'Trench 1'
#       WHEN b.ln_loan_level_user_type = '2_New Applicant'
#            AND DATE_DIFF(DATE(b.ln_appln_submit_datetime), DATE(b.onb_tsa_onboarding_datetime), DAY) > 30 THEN 'Trench 2'
#       WHEN b.ln_loan_level_user_type = '1_Repeat Applicant' THEN 'Trench 3'
#       ELSE NULL
#     END AS trench_category,
#     ROW_NUMBER() OVER (PARTITION BY a.digitalLoanAccountId ORDER BY b.ln_appln_submit_datetime DESC) AS row_num
#   FROM risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy a
#   LEFT JOIN prj-prod-dataplatform.risk_mart.applied_loans_20210701_20250930_trans b
#     ON a.digitalLoanAccountId = b.digitalLoanAccountId
# ) AS source
# ON target.digitalLoanAccountId = source.digitalLoanAccountId
# WHEN MATCHED AND source.row_num = 1 THEN
#   UPDATE SET target.trench_category = source.trench_category;

In [68]:
sq = """ 
select 
  r.customerId customer_id,
  r.digitalLoanAccountId,
  r.apps_score beta_cash_app_score,  
  r.app_cnt_health_and_fitness_ever app_cnt_health_and_fitness_ever, 
  r.app_cnt_shopping_ever app_cnt_shopping_ever,
  r.app_cnt_crypto_ever app_cnt_crypto_ever, 
  r.app_cnt_driver_ever app_cnt_driver_ever,
  r.app_cnt_payday_180d app_cnt_payday_180d, 
  r.app_cnt_gambling_180d app_cnt_gambling_180d,
  r.app_avg_time_bw_installed_mins_3d app_avg_time_bw_installed_mins_3d,
  r.app_median_time_bw_installed_mins_ever app_median_time_bw_installed_mins_3d,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType
from risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy r 
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 1';
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID b01a5739-fff9-4d0a-9ce8-1024bd3341f5 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (386694, 12)


In [69]:
data.columns

Index(['customer_id', 'digitalLoanAccountId', 'beta_cash_app_score',
       'app_cnt_health_and_fitness_ever', 'app_cnt_shopping_ever',
       'app_cnt_crypto_ever', 'app_cnt_driver_ever', 'app_cnt_payday_180d',
       'app_cnt_gambling_180d', 'app_avg_time_bw_installed_mins_3d',
       'app_median_time_bw_installed_mins_3d', 'osType'],
      dtype='object')

In [70]:
feature_column = ['beta_cash_app_score',
       'app_cnt_health_and_fitness_ever', 'app_cnt_shopping_ever',
       'app_cnt_crypto_ever', 'app_cnt_driver_ever', 'app_cnt_payday_180d',
       'app_cnt_gambling_180d', 'app_avg_time_bw_installed_mins_3d',
       'app_median_time_bw_installed_mins_3d']

dfd = transform_data(data, feature_column, a='beta_cash_app_score', modelDisplayName='Beta-Cash-AppScore-Model', tc='Trench 1', subscription_name = 'Cash September 25 Models') 
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386694 entries, 0 to 386693
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   customerId            386694 non-null  int64  
 1   digitalLoanAccountId  386694 non-null  object 
 2   crifApplicationId     386694 non-null  object 
 3   prediction            386694 non-null  float64
 4   start_time            386694 non-null  object 
 5   end_time              386694 non-null  object 
 6   modelDisplayName      386694 non-null  object 
 7   modelVersionId        386694 non-null  object 
 8   calcFeature           386694 non-null  object 
 9   subscription_name     386694 non-null  object 
 10  message_id            386694 non-null  object 
 11  publish_time          386694 non-null  object 
 12  attributes            386694 non-null  object 
 13  trenchCategory        386694 non-null  object 
 14  deviceOs              386694 non-null  object 
dtype

In [71]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=0b1d334e-c2ec-4b40-80bb-7dee86450226>

###### Trench 2

In [72]:
sq = """ 
select 
  r.customerId customer_id,
  r.digitalLoanAccountId,
  r.apps_score beta_cash_app_score,  
  r.app_cnt_health_and_fitness_ever app_cnt_health_and_fitness_ever, 
  r.app_cnt_shopping_ever app_cnt_shopping_ever,
  r.app_cnt_crypto_ever app_cnt_crypto_ever, 
  r.app_cnt_driver_ever app_cnt_driver_ever,
  r.app_cnt_payday_180d app_cnt_payday_180d, 
  r.app_cnt_gambling_180d app_cnt_gambling_180d,
  r.app_avg_time_bw_installed_mins_3d app_avg_time_bw_installed_mins_3d,
  r.app_median_time_bw_installed_mins_ever app_median_time_bw_installed_mins_3d,
    case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType
from risk_mart.applied_quick_loan_new_applicants_20230101_20250930_app_scored_copy r 
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId
where r.trench_category = 'Trench 2';
"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 0c7330ae-2d02-48c6-ba4c-e671be2b564f successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (122895, 12)


In [73]:
feature_column = ['beta_cash_app_score',
       'app_cnt_health_and_fitness_ever', 'app_cnt_shopping_ever',
       'app_cnt_crypto_ever', 'app_cnt_driver_ever', 'app_cnt_payday_180d',
       'app_cnt_gambling_180d', 'app_avg_time_bw_installed_mins_3d',
       'app_median_time_bw_installed_mins_3d']

dfd = transform_data(data, feature_column, a='beta_cash_app_score', modelDisplayName='Beta-Cash-AppScore-Model', tc='Trench 2', subscription_name = 'Cash September 25 Models') 
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122895 entries, 0 to 122894
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   customerId            122895 non-null  int64  
 1   digitalLoanAccountId  122895 non-null  object 
 2   crifApplicationId     122895 non-null  object 
 3   prediction            122895 non-null  float64
 4   start_time            122895 non-null  object 
 5   end_time              122895 non-null  object 
 6   modelDisplayName      122895 non-null  object 
 7   modelVersionId        122895 non-null  object 
 8   calcFeature           122895 non-null  object 
 9   subscription_name     122895 non-null  object 
 10  message_id            122895 non-null  object 
 11  publish_time          122895 non-null  object 
 12  attributes            122895 non-null  object 
 13  trenchCategory        122895 non-null  object 
 14  deviceOs              122895 non-null  object 
dtype

In [74]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=90aa1b89-8fa4-4a24-bc72-e802ae037aba>

###### Trench 3

In [75]:
sq = """   
 select 
  r.customerId customer_id, 
  r.digitalLoanAccountId,
  r.apps_score beta_cash_app_score,
  r.app_cnt_health_and_fitness_ever_binned app_cnt_health_and_fitness_ever,
  r.app_cnt_productivity_ever_binned app_cnt_productivity_ever, 
  r.app_cnt_rated_for_18plus_ever_binned app_cnt_rated_for_18plus_ever,
  r.app_cnt_books_and_reference_ever_binned app_cnt_books_and_reference_ever, 
  r.app_cnt_gaming_180d_binned app_cnt_gaming_180d,
  r.app_cnt_absence_tag_365d_binned app_cnt_absence_tag_365d,
  r.app_last_payday_install_to_apply_days_binned app_last_payday_install_to_apply_days,
  r.app_cnt_absence_tag_365d_binned,
  r.app_cnt_gaming_180d_binned,
  r.app_cnt_productivity_ever_binned,
  r.app_cnt_rated_for_18plus_ever_binned,
  r.app_cnt_health_and_fitness_ever_binned,
  r.app_cnt_books_and_reference_ever_binned,
  r.app_last_payday_install_to_apply_days_binned,
  r.ln_user_type,
  case when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(lmt.osversion_v2, lmt.osVersion)) like '%os%' then 'ios'
    when lower(lmt.deviceType) like '%andro%' then 'android'
    else 'ios' end osType
from risk_mart.b_score_model_applied_loans_cash_20240101_20250930_app_scored r
left join `risk_credit_mis.loan_master_table` lmt on lmt.digitalLoanAccountId = r.digitalLoanAccountId

"""
data = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of the dataframe is:\t {data.shape}")

Job ID 6675c1b1-9dc0-4f80-9129-ad71e69e541e successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is:	 (67403, 19)


In [76]:
feature_column = ['beta_cash_app_score',
       'app_cnt_health_and_fitness_ever', 'app_cnt_productivity_ever',
       'app_cnt_rated_for_18plus_ever', 'app_cnt_books_and_reference_ever',
       'app_cnt_gaming_180d', 'app_cnt_absence_tag_365d',
       'app_last_payday_install_to_apply_days',
       'app_cnt_absence_tag_365d_binned', 'app_cnt_gaming_180d_binned',
       'app_cnt_productivity_ever_binned',
       'app_cnt_rated_for_18plus_ever_binned',
       'app_cnt_health_and_fitness_ever_binned',
       'app_cnt_books_and_reference_ever_binned',
       'app_last_payday_install_to_apply_days_binned', 'ln_user_type']

dfd = transform_data(data, feature_column, a='beta_cash_app_score', modelDisplayName='Beta-Cash-AppScore-Model', tc='Trench 3', subscription_name = 'Cash September 25 Models') 
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,calcFeature,subscription_name,message_id,publish_time,attributes,trenchCategory,deviceOs
0,2294209,1414eb06-0b26-4915-8919-4405b70d8b82,a0931624-e1f0-4698-b201-74da92e3b23c,0.540415,2025-11-28T13:56:59.481110,2025-11-28T13:56:59.481110,Beta-Cash-AppScore-Model,v1,"{""beta_cash_app_score"": 0.540415445062507, ""ap...",Cash September 25 Models,772dc68a-60c8-499d-8f97-192237504d25,2025-11-28T13:56:59.481110,{},Trench 3,android
1,2053840,c60ff04e-601b-4d66-b591-74880234ceec,63ff59cd-118b-41bd-918b-341245b21523,0.431662,2025-11-28T13:56:59.481649,2025-11-28T13:56:59.481649,Beta-Cash-AppScore-Model,v1,"{""beta_cash_app_score"": 0.4316618519910772, ""a...",Cash September 25 Models,983cf0e5-4656-4c34-89d5-b2a6e8138f3a,2025-11-28T13:56:59.481649,{},Trench 3,android
2,2430431,8746782b-b1d3-41da-8775-9a9b8415309a,d4f60f70-8567-45b1-9ffe-3fb444a87dde,0.505247,2025-11-28T13:56:59.482185,2025-11-28T13:56:59.482185,Beta-Cash-AppScore-Model,v1,"{""beta_cash_app_score"": 0.5052472275088205, ""a...",Cash September 25 Models,53abd53c-f9cb-45fd-872f-adf37d832512,2025-11-28T13:56:59.482185,{},Trench 3,android
3,3152521,d9b5d64e-174b-46e6-adfd-765ba305b823,ecc5e569-fe5f-4ef0-9b7c-4d143e0212c3,0.429730,2025-11-28T13:56:59.482185,2025-11-28T13:56:59.482185,Beta-Cash-AppScore-Model,v1,"{""beta_cash_app_score"": 0.42972979974309355, ""...",Cash September 25 Models,8d1fe56d-223e-4dd1-ad5e-38fec038fb3e,2025-11-28T13:56:59.482185,{},Trench 3,android
4,2255547,174535e2-ab04-4f8c-97cb-285caf49b594,f42bb04b-1d91-4901-96b0-712ee57affc8,0.515978,2025-11-28T13:56:59.482185,2025-11-28T13:56:59.482185,Beta-Cash-AppScore-Model,v1,"{""beta_cash_app_score"": 0.5159776064817916, ""a...",Cash September 25 Models,b3fa75d8-cfc1-4ca7-b20f-b4fc46efdb49,2025-11-28T13:56:59.482185,{},Trench 3,android


In [77]:
dfd.info()
dfd['customerId'] = pd.to_numeric(dfd['customerId'], errors='coerce')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67403 entries, 0 to 67402
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   customerId            67403 non-null  object 
 1   digitalLoanAccountId  67403 non-null  object 
 2   crifApplicationId     67403 non-null  object 
 3   prediction            67403 non-null  float64
 4   start_time            67403 non-null  object 
 5   end_time              67403 non-null  object 
 6   modelDisplayName      67403 non-null  object 
 7   modelVersionId        67403 non-null  object 
 8   calcFeature           67403 non-null  object 
 9   subscription_name     67403 non-null  object 
 10  message_id            67403 non-null  object 
 11  publish_time          67403 non-null  object 
 12  attributes            67403 non-null  object 
 13  trenchCategory        67403 non-null  object 
 14  deviceOs              67403 non-null  object 
dtypes: float64(1), obje

In [78]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dfd, table_id, job_config=job_config)
job.result() 

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=fc2eb287-0371-4d3d-a24e-68334fa6986f>

# End